In [ ]:
# modulo1_interface.py

"""
Módulo 1 - Interface Básica e Ambiente
---------------------------------------------------------
Responsabilidades:
1. Importar as bibliotecas essenciais (NumPy, Pandas, SciPy, Matplotlib, ipywidgets, etc.).
2. Declarar constantes globais, como a constante universal dos gases (R) – fundamental nos cálculos de sinterização.
3. Definir funções utilitárias para a interface (criação de títulos, exibição de mensagens/erros, widgets de layout).
4. Preparar recursos e layouts que serão compartilhados por outros módulos para a construção de curvas mestras e curvas de sinterização.
"""

# ------------------------------
# # Importação das bibliotecas essenciais
# ------------------------------
import os
import base64
from pathlib import Path
from uuid import uuid4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from dataclasses import dataclass, field
import pandas as pd
from typing import List, Any

@dataclass
class SinteringDataRecord:
    """
    Uma estrutura para armazenar um conjunto de dados de sinterização e seus metadados.
    Isso substitui a necessidade de nomes de colunas complexos.
    """
    ensaio_id: int
    Ea: float
    tipo_dado_y: str
    df: pd.DataFrame
    metadata: dict = field(default_factory=dict)


# Importa funções do SciPy para processamento dos dados, filtragem e ajuste de curvas (cruciais para sinterização)
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
try:
    from scipy.integrate import cumtrapz
except ImportError:
    # Implementação alternativa de cumtrapz caso não esteja disponível
    def cumtrapz(y, x=None, initial=0):
        y = np.asarray(y)
        if x is None:
            x = np.arange(len(y))
        else:
            x = np.asarray(x)
        res = [initial]
        for i in range(1, len(y)):
            trap = (y[i-1] + y[i]) * (x[i] - x[i-1]) / 2.0
            res.append(res[-1] + trap)
        return np.array(res)

from scipy.optimize import curve_fit

# Importação dos widgets para interfaces interativas no Jupyter
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# ------------------------------
# # Declaração de constantes globais
# ------------------------------
R = 8.314  # Constante universal dos gases (J/mol.K) – usada para cálculos termodinâmicos na sinterização


# ------------------------------
#  Funções utilitárias para a interface
# ------------------------------


def add_suffix_once(col: str, suffix: str) -> str:
    """Retorna a string 'col' com o 'suffix' adicionado, se ela ainda não o possuir."""
    return col if col.endswith(suffix) else f"{col}{suffix}"


def criar_titulo(texto: str, nivel: int = 2) -> widgets.HTML:
    """
    Cria um título HTML para a interface, padronizando os cabeçalhos de seção.

    Parâmetros:
      texto (str): Texto do título.
      nivel (int): Nível do cabeçalho (1 a 6), definindo sua importância visual.

    Retorna:
      widgets.HTML: Título formatado para a interface.

    Contexto:
      Os títulos ajudam a dividir e identificar as etapas do processamento de sinterização,
      como a construção de curvas mestras.
    """
    return widgets.HTML(f"<h{nivel}>{texto}</h{nivel}>")

def exibir_mensagem(msg: str):
    """
    Exibe uma mensagem informativa na interface com formatação em azul.

    Parâmetros:
      msg (str): Mensagem a ser exibida.

    Contexto:
      Indica progresso ou informações relevantes durante a análise das curvas de sinterização.
    """
    display(HTML(f"<p style='color:blue;'>{msg}</p>"))

def exibir_erro(msg: str):
    """
    Exibe uma mensagem de erro na interface com destaque em vermelho e negrito.

    Parâmetros:
      msg (str): Mensagem de erro.

    Contexto:
      Alerta o usuário sobre problemas críticos durante o processamento dos dados de sinterização.
    """
    display(HTML(f"<p style='color:red;font-weight:bold;'>{msg}</p>"))

def criar_caixa_colapsavel(titulo: str, conteudo):
    """
    Cria uma caixa colapsável para organizar seções da interface.

    Parâmetros:
      titulo (str): Título da caixa.
      conteudo: Widget ou lista de widgets a ser exibido ao expandir a caixa.

    Retorna:
      widgets.Accordion: Caixa colapsável formatada.

    Contexto:
      Facilita a organização visual da interface, especialmente quando há várias etapas na
      construção e análise de curvas de sinterização.
    """
    acc = widgets.Accordion(children=[conteudo])
    acc.set_title(0, titulo)
    return acc

# ------------------------------
# 4) Optional Download Utilities
# ------------------------------

def gerar_link_download(df: pd.DataFrame,
                        nome_arquivo: str = "dados.xlsx") -> widgets.HTML:
    """
    Salva o DataFrame em /tmp e devolve um link que o Voilá serve via /files.
    Funciona mesmo para arquivos grandes.
    """
    # gera nome único para evitar colisão em /tmp
    uid = uuid4().hex[:6]
    stem = Path(nome_arquivo).stem
    final = f"{stem}_{uid}.xlsx"
    caminho = Path("/tmp") / final

    # escreve no disco
    with pd.ExcelWriter(caminho, engine="openpyxl") as writer:
        df.to_excel(writer, index=False)

    # link para /files/tmp/NOME que o Jupyter/Voila já serve como estático
    html = f"""
    <a href="/files/tmp/{final}" download="{final}" target="_blank">
      Clique para baixar {final}
    </a>
    """
    return widgets.HTML(html)


# =====================================================================
# ADICIONE ESTA CLASSE NO FINAL DO SEU ARQUIVO modulo1_interface.py
# =====================================================================

class EaSelectionWidget:
    """
    Um widget reutilizável para selecionar Energias de Ativação (Ea)
    nos modos 'Discretas' ou 'Intervalo'. Encapsula toda a lógica da UI
    e o método para obter a lista de energias selecionadas.
    """
    def __init__(self):
        self._build_ui()

    def _build_ui(self):
        # --- Widgets Internos ---
        self.radio_energy_choice = widgets.RadioButtons(
            options=[("Discretas", "discretas"), ("Intervalo", "intervalo")],
            value="discretas", description="Modo de Seleção de Ea:"
        )

        # Widgets para o modo "Discretas"
        self.dropdown_num_energies = widgets.Dropdown(
            options=[(f"{i} energias", i) for i in range(1, 11)],
            value=3, description="Quantidade:"
        )
        self.btn_energy_add = widgets.Button(description="Gerar Campos", button_style='info')
        self.energies_box = widgets.VBox([]) # Container para os campos de Ea

        # Widgets para o modo "Intervalo"
        self.float_min = widgets.FloatText(value=100.0, description="Ea Mín (kJ/mol):")
        self.float_max = widgets.FloatText(value=500.0, description="Ea Máx (kJ/mol):")
        self.num_points = widgets.IntText(value=26, description="Nº de Pontos:")

        # --- Containers de Layout ---
        self.discretas_container = widgets.VBox([
            self.dropdown_num_energies, self.btn_energy_add, self.energies_box
        ])
        self.intervalo_container = widgets.VBox([
            self.float_min, self.float_max, self.num_points
        ])

        # Container principal que alterna os modos
        self.main_container = widgets.VBox([self.discretas_container])

        # --- Conecta Callbacks ---
        self.radio_energy_choice.observe(self._on_energy_choice_changed, names='value')
        self.btn_energy_add.on_click(self._on_add_energies)

        # --- Widget Final (Público) ---
        # Este é o widget que será exibido por outros módulos
        self.ui = widgets.VBox([
            widgets.HTML("<b>Definir Energias de Ativação (Ea) para Análise</b>"),
            self.radio_energy_choice,
            self.main_container
        ])

    def _on_energy_choice_changed(self, change):
        """Alterna a exibição entre os containers de modo."""
        if change['new'] == "discretas":
            self.main_container.children = [self.discretas_container]
        else:
            self.main_container.children = [self.intervalo_container]

    def _on_add_energies(self, b):
        """Gera os campos de texto para o modo 'Discretas'."""
        n = self.dropdown_num_energies.value
        self.energies_box.children = [
            widgets.FloatText(value=round(100.0 + 50 * i, 1), description=f"Ea {i+1} (kJ/mol):") for i in range(n)
        ]
        # Gera 3 campos por padrão na primeira vez
        if not self.energies_box.children:
            self._on_add_energies(None)

    def get_ea_list(self) -> List[float]:
        """
        Método público que retorna a lista de energias de ativação
        com base na seleção atual da interface.
        """
        if self.radio_energy_choice.value == "discretas":
            # Gera os campos se estiverem vazios
            if not self.energies_box.children:
                self._on_add_energies(None)

            values = [ft.value for ft in self.energies_box.children if ft.value is not None and ft.value > 0]
            if not values:
                exibir_erro("Nenhuma energia válida foi informada no modo 'Discretas'.")
                return []
            return sorted(set(values))
        else: # Modo Intervalo
            ea_min, ea_max = self.float_min.value, self.float_max.value
            n_pts = self.num_points.value
            if ea_max <= ea_min or n_pts < 2:
                exibir_erro("No modo 'Intervalo', Ea Máx > Ea Mín e Nº de Pontos deve ser >= 2.")
                return []
            return list(np.linspace(ea_min, ea_max, n_pts))

# ------------------------------
# Fim do Módulo 1
# ------------------------------

In [ ]:
# modulo2_importacao.py (VERSÃO FINAL AJUSTADA)

import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from io import BytesIO

# Funções e classes que devem estar definidas no Módulo 1 ou em uma célula anterior
# Ex: criar_titulo, exibir_mensagem, exibir_erro, DataHistory, add_suffix_once

class Modulo2Importacao:
    """
    Classe principal para o Módulo 2 – Importação de Dados.

    Gerencia:
    - A seleção do número de testes.
    - O upload dos arquivos .xlsx.
    - O mapeamento e renomeação das colunas para garantir que existam as informações necessárias.
    - A conversão dos dados de retração para densidade.
    """
    def __init__(self):
        self.n_ensaios = 1
        self.dfs_ensaios = [None, None, None, None]  # Até 4 testes
        self.file_uploads = []  # Widgets de upload e mapeamento para cada teste
        # Histórico para rastrear os estados dos dados
        self.history = DataHistory()
        self._build_widgets()

    def _build_widgets(self):
        # Cria título e descrição introdutória do módulo
        self.title_label = criar_titulo("Módulo 2 – Importação e Reconhecimento de Dados", nivel=2)

        # Texto Introdutório para o Módulo
        intro_text = """
        <div style="font-family: Arial, sans-serif; line-height: 1.5;">
            <p>
                Neste módulo, os dados experimentais serão carregados a partir de arquivos .xlsx.
                É fundamental que os dados contenham informações de Tempo, Temperatura e pelo menos um
                parâmetro relacionado à sinterização (ex.: DensidadePct ou Retração).
            </p>
            <p>
                Estes dados serão processados para possibilitar a construção das curvas mestras de sinterização.
            </p>
        </div>
        """
        self.intro_widget = widgets.HTML(value=intro_text)

        # Widget para selecionar o número de testes a serem carregados
        self.dropdown_ensaios = widgets.Dropdown(
            options=[("1 Teste", 1), ("2 Testes", 2), ("3 Testes", 3), ("4 Testes", 4)],
            value=1,
            description="Número de testes:"
        )
        self.btn_confirm_ensaios = widgets.Button(
            description="Confirme o número de Testes",
            button_style='success'
        )
        self.btn_confirm_ensaios.on_click(self._confirmar_num_ensaios)

        self.upload_box = widgets.VBox([])
        self.out = widgets.Output()

        self.main_ui = widgets.VBox([
            self.title_label,
            self.intro_widget,
            widgets.HTML("<b>Selecione quantos ensaios você selecionará:</b>"),
            widgets.HBox([self.dropdown_ensaios, self.btn_confirm_ensaios]),
            self.upload_box,
            self.out
        ])

    def display(self):
        display(self.main_ui)

    def _confirmar_num_ensaios(self, b):
        with self.out:
            clear_output()
        self.n_ensaios = self.dropdown_ensaios.value
        exibir_mensagem(f"{self.n_ensaios} Testes serão carregados.")

        self.upload_box.children = []
        self.file_uploads = []

        for i in range(self.n_ensaios):
            uploader = widgets.FileUpload(accept='.xlsx', multiple=False)
            btn_load = widgets.Button(description=f"Carregar Teste {i+1}", button_style='info')
            out_local = widgets.Output()

            map_title = widgets.HTML(f"<b>Mapear Colunas (Teste {i+1})</b>")
            map_box = widgets.VBox([])
            map_confirm_btn = widgets.Button(description=f"Confirmar Mapeamento {i+1}", button_style='success')
            map_confirm_btn.layout.display = 'none'

            conv_panel = widgets.VBox([
                widgets.HTML("<b>Parâmetros de Conversão (Retração -> Densidade)</b>"),
                widgets.FloatText(value=5.0, description="Tamanho Inicial (mm):"),
                widgets.FloatText(value=55.0, description="Densidade Inicial (%):"),
                widgets.FloatText(value=90.0, description="Densidade Final (%):"),
                widgets.FloatText(value=6.0, description="Densidade Teórica (g/cm³):")
            ])
            conv_params = {
                "tamanho_inicial": conv_panel.children[1],
                "dens_inicial": conv_panel.children[2],
                "dens_final": conv_panel.children[3],
                "dens_teorica": conv_panel.children[4]
            }

            btn_convert = widgets.Button(description=f"Converter Teste {i+1}", button_style='warning')
            btn_convert.on_click(self._on_convert(i))

            self.file_uploads.append({
                "uploader": uploader, "btn_load": btn_load, "out_local": out_local,
                "map_title": map_title, "map_box": map_box, "map_confirm_btn": map_confirm_btn,
                "mapping_dds": [], "conv_params": conv_params, "btn_convert": btn_convert
            })

            btn_load.on_click(self._on_load_data(i))
            map_confirm_btn.on_click(self._on_confirm_mapping(i))

            vbox_ensaio = widgets.VBox([
                widgets.HTML(f"<hr><b>Teste {i+1}</b>"),
                uploader, btn_load, out_local,
                map_title, map_box, map_confirm_btn,
                conv_panel, btn_convert
            ])
            self.upload_box.children += (vbox_ensaio,)

    def _on_load_data(self, idx):
        def callback(b):
            w = self.file_uploads[idx]
            out_local = w["out_local"]
            with out_local:
                clear_output()
                uploader = w["uploader"]
                if not uploader.value:
                    exibir_erro(f"Teste {idx+1}: Nenhum arquivo selecionado!")
                    return
                try:
                    if isinstance(uploader.value, dict):
                        file_meta = list(uploader.value.values())[0]
                    elif isinstance(uploader.value, (list, tuple)) and len(uploader.value) > 0:
                        file_meta = uploader.value[0]
                    else:
                        raise ValueError(f"Formato inesperado do uploader.")

                    file_data = file_meta.get('content')
                    if not file_data:
                        raise ValueError("O conteúdo do arquivo está vazio.")

                    df = pd.read_excel(BytesIO(file_data))
                    self.dfs_ensaios[idx] = df
                    exibir_mensagem(f"Teste {idx+1}: Arquivo carregado. Colunas: {df.columns.tolist()}")
                    self._create_mapping_ui_for_ensaio(idx, df)

                except Exception as e:
                    exibir_erro(f"Teste {idx+1}: Erro ao ler o arquivo - {e}")
        return callback

    def _create_mapping_ui_for_ensaio(self, idx, df):
        w = self.file_uploads[idx]
        map_box = w["map_box"]
        map_confirm_btn = w["map_confirm_btn"]
        map_box.children = []
        map_confirm_btn.layout.display = 'none'

        options = [
            ('Ignorar', 'ignore'), ('Tempo (s)', 'Time_s'), ('Tempo (min)', 'Time_min'),
            ('Tempo (h)', 'Time_h'), ('Temperatura (°C)', 'Temperature_C'),
            ('Retração Absoluta', 'RetracaoAbs'), ('Retração Percentual', 'RetracaoPct'),
            ('Retração Relativa', 'RetracaoRel'), ('Densidade Absoluta', 'DensidadeAbs'),
            ('Densidade Percentual', 'DensidadePct')
        ]
        children_dd = [widgets.Dropdown(options=options, value='ignore', description=col) for col in df.columns]
        map_box.children = children_dd
        map_confirm_btn.layout.display = ''
        w["mapping_dds"] = children_dd

# =================================================================================
# MÉTODO CORRIGIDO para a classe Modulo2Importacao
# =================================================================================

    def _on_confirm_mapping(self, idx):
        def callback(b):
            w = self.file_uploads[idx]
            out_local = w["out_local"]
            with out_local:
                clear_output()
                df = self.dfs_ensaios[idx]
                if df is None:
                    exibir_erro(f"Teste {idx+1}: DataFrame está vazio.")
                    return

                dds = w["mapping_dds"]
                rename_dict = {dd.description.strip(): dd.value for dd in dds if dd.value != 'ignore'}

                df_temp = df[[col for col in rename_dict]].copy()
                df_temp.rename(columns=rename_dict, inplace=True)

                # --- Lógica de conversão de tempo ---
                def get_single_column_by_prefix(df_in, prefix):
                    matching_cols = [col for col in df_in.columns if col.startswith(prefix)]
                    if len(matching_cols) > 1:
                        exibir_erro(f"Mapeamento ambíguo: {len(matching_cols)} colunas mapeadas como '{prefix}*'. Mapeie apenas uma.")
                        return None, True
                    return (matching_cols[0] if matching_cols else None), False

                is_error = False
                time_s_col, error = get_single_column_by_prefix(df_temp, 'Time_s')
                is_error |= error
                time_min_col, error = get_single_column_by_prefix(df_temp, 'Time_min')
                is_error |= error
                time_h_col, error = get_single_column_by_prefix(df_temp, 'Time_h')
                is_error |= error

                if is_error: return

                if time_s_col:
                    exibir_mensagem("Coluna 'Time_s' encontrada.")
                elif time_min_col:
                    exibir_mensagem(f"Convertendo '{time_min_col}' para segundos...")
                    df_temp['Time_s'] = df_temp[time_min_col] * 60.0
                elif time_h_col:
                    exibir_mensagem(f"Convertendo '{time_h_col}' para segundos...")
                    df_temp['Time_s'] = df_temp[time_h_col] * 3600.0

                # --- Validação de colunas ---
                missing = []
                if not any(c.startswith('Time_s') for c in df_temp.columns): missing.append('Tempo')
                if not any(c.startswith('Temperature_C') for c in df_temp.columns): missing.append('Temperatura')
                if not any(c.startswith('DensidadePct') or c.startswith('Retracao') for c in df_temp.columns):
                    missing.append("Densidade ou Retração")

                if missing:
                    exibir_erro(f"Teste {idx+1}: Colunas obrigatórias faltantes: {', '.join(missing)}.")
                    return

                # =================================================================
                # LINHA CRÍTICA ADICIONADA AQUI
                # Adiciona o sufixo do ensaio a TODAS as colunas ANTES de salvar.
                # =================================================================
                df_temp.columns = [add_suffix_once(col, f"_e{idx+1}") for col in df_temp.columns]

                # Agora o DataFrame é salvo com os nomes corretos (ex: RetracaoPct_e1)
                self.dfs_ensaios[idx] = df_temp

                exibir_mensagem(f"Teste {idx+1}: Mapeamento completo. Colunas Finais: {df_temp.columns.tolist()}")

                if not hasattr(self, 'history'): self.history = DataHistory()
                self.history.push(self.dfs_ensaios[idx], module_name=f"Module 2: Mapping Test {idx+1}")
        return callback

# =================================================================================
# MÉTODO CORRIGIDO para a classe Modulo2Importacao
# =================================================================================

    def _on_convert(self, idx):
        """
        Callback para converter os dados de retração em densidade para o ensaio idx.
        Versão corrigida para procurar os nomes de coluna com o sufixo do ensaio.
        """
        def callback(b):
            w = self.file_uploads[idx]
            out_local = w["out_local"]
            df = self.dfs_ensaios[idx]

            with out_local:
                clear_output()
                if df is None or df.empty:
                    exibir_erro(f"Ensaio {idx+1}: DF vazio, não posso converter.")
                    return

                conv = w["conv_params"]
                di = conv["dens_inicial"].value
                df_final = conv["dens_final"].value
                lo = conv["tamanho_inicial"].value

                # Nome da coluna de densidade que queremos criar
                final_dens_col = f'DensidadePct_e{idx+1}'

                if final_dens_col in df.columns:
                    exibir_mensagem(f"Ensaio {idx+1}: Coluna '{final_dens_col}' já existe. Conversão não necessária.")
                    return

                df_temp = df.copy()

                try:
                    # --- LÓGICA DE CONVERSÃO CORRIGIDA ---
                    # Para cada tipo de retração, procuramos pelo nome da coluna JÁ COM O SUFIXO

                    col_retracao_rel = f'RetracaoRel_e{idx+1}'
                    col_retracao_pct = f'RetracaoPct_e{idx+1}'
                    col_retracao_abs = f'RetracaoAbs_e{idx+1}'

                    # Converte de Retração Relativa
                    if col_retracao_rel in df_temp.columns:
                        exibir_mensagem(f"Convertendo '{col_retracao_rel}'...")
                        df_temp[final_dens_col] = di * (1 / (1 + df_temp[col_retracao_rel]))**3
                        df_temp.drop(columns=[col_retracao_rel], inplace=True, errors='ignore')

                    # Converte de Retração Percentual
                    elif col_retracao_pct in df_temp.columns:
                        exibir_mensagem(f"Convertendo '{col_retracao_pct}'...")
                        df_temp['RetracaoRel_temp'] = df_temp[col_retracao_pct] / 100.0
                        df_temp[final_dens_col] = di * (1 / (1 + df_temp['RetracaoRel_temp']))**3
                        df_temp.drop(columns=[col_retracao_pct, 'RetracaoRel_temp'], inplace=True, errors='ignore')

                    # Converte de Retração Absoluta
                    elif col_retracao_abs in df_temp.columns:
                        exibir_mensagem(f"Convertendo '{col_retracao_abs}'...")
                        if lo <= 0:
                            raise ValueError("Tamanho Inicial (lo) deve ser maior que zero para converter Retração Absoluta.")
                        df_temp['RetracaoRel_temp'] = df_temp[col_retracao_abs] / lo
                        df_temp[final_dens_col] = di * (1 / (1 + df_temp['RetracaoRel_temp']))**3
                        df_temp.drop(columns=[col_retracao_abs, 'RetracaoRel_temp'], inplace=True, errors='ignore')

                    # Se após todas as verificações, a coluna de densidade não foi criada, então há um erro.
                    if final_dens_col not in df_temp.columns:
                        raise ValueError("Nenhuma coluna de retração aplicável encontrada para conversão.")

                    # Ajuste linear para a faixa de densidade desejada
                    curr_min = df_temp[final_dens_col].min()
                    curr_max = df_temp[final_dens_col].max()
                    if curr_max > curr_min:
                        df_temp[final_dens_col] = di + (df_temp[final_dens_col] - curr_min) * (df_final - di) / (curr_max - curr_min)
                    else: # Caso a densidade seja constante
                        df_temp[final_dens_col] = di

                    # Atualiza o DataFrame do ensaio com os dados convertidos
                    self.dfs_ensaios[idx] = df_temp
                    exibir_mensagem(f"Ensaio {idx+1}: Conversão concluída. Colunas finais: {df_temp.columns.tolist()}")

                except Exception as e:
                    exibir_erro(f"Ensaio {idx+1}: Erro na conversão - {e}")

        return callback

    def get_dfs(self):
        return self.dfs_ensaios[:self.n_ensaios]

In [ ]:
# modulo3_filtrorecorte.py

"""
Módulo 3 - Recorte de Dados, Ajuste de Tempo, Visualização e Filtragem
---------------------------------------------------------
Responsabilidades:
1. Receber os DataFrames processados no Módulo 2 e prepará-los para análise.
2. Permitir o ajuste do tempo (por exemplo, zerar o tempo para alinhar os dados) e o corte de intervalos de interesse.
3. Aplicar filtros e interpolação (como Savitzky-Golay, Orlandini-Araujo, interpolação linear/cúbica) para suavizar os dados.
4. Visualizar as curvas experimentais (Tempo vs. Densidade, Temperatura vs. Densidade, etc.), essenciais para a construção das curvas de sinterização.
5. Registrar o estado dos dados utilizando um histórico para possibilitar “undo” ou revisões posteriores.
"""


import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d

# Supondo que no Módulo 1 você já tenha exibir_mensagem, exibir_erro, gerar_link_download, etc.
# Aqui coloco stubs para funcionar como exemplo.
def exibir_mensagem(msg):
    print(f"[INFO] {msg}")

def exibir_erro(msg):
    print(f"[ERRO] {msg}")

def gerar_link_download(df, nome_arquivo="dados.xlsx"):
    """
    Exemplo de função que gera link para download de um DataFrame em Excel.
    Você pode adaptar caso já tenha uma implementação.
    """
    import base64
    from io import BytesIO
    out_buf = BytesIO()
    with pd.ExcelWriter(out_buf, engine='openpyxl') as writer:
        df.to_excel(writer, index=False)
    out_buf.seek(0)
    b64 = base64.b64encode(out_buf.read()).decode()
    link_html = f"""
    <a download="{nome_arquivo}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}">
       Clique aqui para baixar: {nome_arquivo}
    </a>
    """
    return widgets.HTML(link_html)

def orlandini_araujo_filter(df, bin_size=10):
    # Procura as colunas por prefixo
    time_col = next((col for col in df.columns if col.startswith('Time_s')), None)
    temp_col = next((col for col in df.columns if col.startswith('Temperature_C')), None)
    dens_col = next((col for col in df.columns if col.startswith('DensidadePct')), None)

    if not (time_col and temp_col and dens_col):
        raise ValueError("Faltam colunas (Time_s, Temperature_C, DensidadePct) p/ Orlandini-Araujo.")

    dfc = df.copy()
    dfc['bin'] = np.floor(dfc[time_col] / bin_size).astype(int)
    grouped = dfc.groupby('bin').agg({
        time_col: 'mean',
        temp_col: 'mean',
        dens_col: 'mean'
    }).reset_index(drop=True)
    return grouped



class Modulo3Recorte:
    """
    Classe responsável pelo recorte e filtragem dos dados experimentais.

    Processa os DataFrames oriundos do Módulo 2 para realizar:
      - Ajuste do tempo (por exemplo, zerar o tempo inicial).
      - Corte de intervalos de tempo relevantes para a análise de sinterização.
      - Aplicação de filtros e interpolação para melhorar a qualidade dos dados.

    Esses passos são fundamentais para a construção das curvas mestras de sinterização.
    """

    def __init__(self, dfs_ensaios):
        # Filtra DF nulos/vazios
        self.dfs_ensaios = [df for df in dfs_ensaios if df is not None and not df.empty]
        self.n_ensaios = len(self.dfs_ensaios)

        # Área de saída (logs gerais do módulo)
        self.out = widgets.Output()
        # Cria uma instância do histórico para este módulo
        self.history = DataHistory()
        # Constrói interface
        self._build_ui()

    def _build_ui(self):
        if self.n_ensaios < 1:
            self.main_ui = widgets.HTML("<b>Nenhum ensaio disponível para Módulo 3.</b>")
            return

        # Cria um painel para cada ensaio
        children_panels = []
        for i in range(self.n_ensaios):
            panel = self._build_ensaio_panel(i)
            children_panels.append(panel)

        # Usa Accordion para exibir cada ensaio
        self.tabs = widgets.Accordion(children=children_panels)
        for i in range(self.n_ensaios):
            self.tabs.set_title(i, f"Ensaio {i+1}")

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Módulo 3 - Recorte e Filtragem</h3>"),
            widgets.HTML("Ajuste de tempo, exibição de curvas, corte de intervalo e filtragem/interpolação."),
            self.tabs,
            self.out
        ])

    def _build_ensaio_panel(self, idx):
        """
        Cria widgets para:
          - Ajuste de Tempo (Zerar/Não Zerar)
          - Plot de Curvas
          - Slider de Corte
          - Filtro/Interpolação
        Organiza o layout em duas colunas:
          (esquerda) controles e logs
          (direita) gráfico
        """
        # Área de saída para logs específicos do ensaio
        vbox_out = widgets.Output()

        # Área de saída para o gráfico
        fig_out = widgets.Output()

        # 1) Radio: Zerar Tempo
        radio_zerar = widgets.RadioButtons(
            options=[("Não Zerar", "nao"), ("Zerar Tempo", "sim")],
            description="Zerar Tempo?",
            value="nao"
        )
        btn_aplicar_zerar = widgets.Button(description="Aplicar Ajuste Tempo", button_style='warning')

        # 2) Slider de Corte (Time_s)
        tmin, tmax = self._get_time_range(idx)
        slider_cut = widgets.FloatRangeSlider(
            value=(tmin, tmax),
            min=tmin,
            max=tmax,
            step=1.0,
            description='Corte Tempo(s)'
        )
        btn_cut = widgets.Button(description="Aplicar Corte", button_style='warning')

        # 3) Botão Plot
        btn_plot = widgets.Button(description="Exibir Curvas", button_style='info')

        # 3b) Botão Exportar Excel
        btn_export = widgets.Button(description="Exportar Excel", button_style='info')

        def on_export(b):
            df_to_export = self.dfs_ensaios[idx]
            with vbox_out:               # vbox_out é o Output deste ensaio
                clear_output()
                if df_to_export is None or df_to_export.empty:
                    exibir_erro(f"Ensaio {idx+1}: DF vazio, nada a exportar.")
                    return
                link = gerar_link_download(df_to_export,
                                          nome_arquivo=f"Ensaio_{idx+1}_Ajustado.xlsx")
                display(link)

        btn_export.on_click(on_export)

        # 4) Filtro/Interpol
        filter_dropdown = widgets.Dropdown(
            options=[
                ('Nenhum', 'none'),
                ('Orlandini-Araujo', 'orlandini'),
                ('Savitzky-Golay', 'savgol'),
                ('Interpolação Linear', 'linear'),
                ('Interpolação Cúbica', 'cubic')
            ],
            value='none',
            description='Filtro:'
        )
        bin_size_widget = widgets.FloatText(value=10.0, description="Bin(s):")
        bin_size_widget.layout.display = 'none'
        n_points_widget = widgets.IntText(value=200, description="N Points:")
        n_points_widget.layout.display = 'none'

        # Observa mudança no dropdown
        filter_dropdown.observe(lambda c: self._on_filter_change(c, bin_size_widget, n_points_widget), 'value')

        btn_filter = widgets.Button(description="Aplicar Filtro/Interp.", button_style='warning')

        # =============== Callbacks ===============

        def on_aplicar_zerar(b):
            with vbox_out:
                clear_output()
            dfi = self.dfs_ensaios[idx]
            if dfi is None or dfi.empty:
                with vbox_out:
                    exibir_erro(f"Ensaio {idx+1}: DF vazio, não posso ajustar tempo.")
                return

            # Encontra a coluna de tempo (base "Time_s")
            time_cols = [col for col in dfi.columns if col.startswith('Time_s')]
            if not time_cols:
                exibir_erro(f"Ensaio {idx+1}: Coluna de tempo não encontrada.")
                return
            time_col = time_cols[0]

            if radio_zerar.value == "sim":
                min_t_df = dfi[time_col].min()
                self.dfs_ensaios[idx][time_col] = dfi[time_col] - min_t_df
                exibir_mensagem(f"Ensaio {idx+1}: Tempo zerado (subtraído {min_t_df}).")
            else:
                exibir_mensagem(f"Ensaio {idx+1}: Tempo mantido (não zerado).")

            # Reordena e atualiza o slider (adaptando para usar o valor da coluna encontrada)
            self.dfs_ensaios[idx].sort_values(time_col, inplace=True)
            self.dfs_ensaios[idx].reset_index(drop=True, inplace=True)
            self._update_slider_range(idx, slider_cut)

            # Salva o estado após o ajuste de tempo
            self.history.push(self.dfs_ensaios[idx], module_name=f"Module 3: Ajuste de Tempo Teste {idx+1}")




        btn_aplicar_zerar.on_click(on_aplicar_zerar)

        def on_cut(b):
            with vbox_out:
                clear_output()
            dfi = self.dfs_ensaios[idx]
            if dfi is None or dfi.empty:
                exibir_erro(f"Ensaio {idx+1}: DF vazio, não posso cortar.")
                return
            # Busca a coluna de tempo
            time_cols = [col for col in dfi.columns if col.startswith('Time_s')]
            if not time_cols:
                exibir_erro(f"Ensaio {idx+1}: Coluna de tempo não encontrada.")
                return
            time_col = time_cols[0]
            min_t, max_t = slider_cut.value
            df_cut = dfi[(dfi[time_col] >= min_t) & (dfi[time_col] <= max_t)].copy()
            kept = len(df_cut)
            total = len(dfi)
            if kept < 1:
                exibir_erro(f"Ensaio {idx+1}: Corte resultou em DF vazio. Intervalo [{min_t}, {max_t}].")
                return
            df_cut.sort_values(time_col, inplace=True)
            df_cut.reset_index(drop=True, inplace=True)
            self.dfs_ensaios[idx] = df_cut
            exibir_mensagem(f"Ensaio {idx+1}: Corte [{min_t}, {max_t}] (s). Mantidos {kept} de {total} pontos.")
            self._update_slider_range(idx, slider_cut)

            # Salva estado após corte
            self.history.push(self.dfs_ensaios[idx], module_name=f"Module 3: Corte de Intervalo Test {idx+1}")



        btn_cut.on_click(on_cut)

        def on_plot(b):
            with vbox_out:
                clear_output()
            dfi = self.dfs_ensaios[idx]
            if dfi is None or dfi.empty:
                exibir_erro(f"Ensaio {idx+1}: DF vazio, não posso proceder.")
                return

            # Busca as colunas obrigatórias dinamicamente
            time_col = next((col for col in dfi.columns if col.startswith('Time_s')), None)
            temp_col = next((col for col in dfi.columns if col.startswith('Temperature_C')), None)
            dens_col = next((col for col in dfi.columns if col.startswith('DensidadePct')), None)

            if not (time_col and temp_col and dens_col):
                missing = [c for c in ['Time_s', 'Temperature_C', 'DensidadePct'] if next((col for col in dfi.columns if col.startswith(c)), None) is None]
                exibir_erro(f"Ensaio {idx+1}: Faltam as colunas obrigatórias: {missing}")
                return

            # Plot usando as colunas encontradas
            with fig_out:
                fig, axs = plt.subplots(1, 3, figsize=(15,4))
                fig.suptitle(f"Ensaio {idx+1} - Visualização")
                axs[0].plot(dfi[time_col], dfi[dens_col], 'o-')
                axs[0].set_xlabel("Tempo (s)")
                axs[0].set_ylabel("Densidade (%)")

                axs[1].plot(dfi[temp_col], dfi[dens_col], 'o-', color='red')
                axs[1].set_xlabel("Temperatura (°C)")
                axs[1].set_ylabel("Densidade (%)")

                axs[2].plot(dfi[time_col], dfi[temp_col], 'o-', color='green')
                axs[2].set_xlabel("Tempo (s)")
                axs[2].set_ylabel("Temperatura (°C)")

                plt.tight_layout()
                display(fig)
                plt.close(fig)


        btn_plot.on_click(on_plot)

        def on_apply_filter(b):
            with vbox_out:
                clear_output()

            dfi = self.dfs_ensaios[idx]
            if dfi is None or dfi.empty:
                exibir_erro(f"Ensaio {idx+1}: DF vazio, não posso filtrar/interpolar.")
                return

            # Procura a coluna de densidade
            dens_col = next((col for col in dfi.columns if col.startswith('DensidadePct')), None)
            if not dens_col:
                exibir_erro(f"Ensaio {idx+1}: Falta 'DensidadePct'. Não posso filtrar.")
                return

            dfi = dfi.copy()
            method = filter_dropdown.value
            before = len(dfi)
            dfi = dfi[(dfi[dens_col] >= 0) & (dfi[dens_col] <= 100)].copy()
            after = len(dfi)
            removed = before - after
            exibir_mensagem(f"Ensaio {idx+1}: Removidos {removed} pontos fora de [0..100]%.")

            if len(dfi) < 2:
                exibir_erro(f"Ensaio {idx+1}: DF insuficiente após remoção. Tamanho={len(dfi)}.")
                return

            try:
                if method == 'none':
                    exibir_mensagem(f"Ensaio {idx+1}: Nenhum filtro aplicado.")
                elif method == 'orlandini':
                    bin_size = bin_size_widget.value
                    if len(dfi) < bin_size:
                        exibir_erro(f"Ensaio {idx+1}: Número de pontos ({len(dfi)}) menor que o tamanho do bin ({bin_size}).")
                        return
                    dfi = orlandini_araujo_filter(dfi, bin_size=bin_size)
                    exibir_mensagem(f"Ensaio {idx+1}: Filtro Orlandini (bin={bin_size}) aplicado.")
                elif method == 'savgol':
                    if len(dfi) < 5:
                        exibir_erro(f"Ensaio {idx+1}: Poucos pontos p/ Savitzky-Golay.")
                        return
                    w = min(11, (len(dfi)//2)*2+1)
                    dfi[dens_col] = savgol_filter(dfi[dens_col], w, 2)
                    exibir_mensagem(f"Ensaio {idx+1}: Savitzky-Golay aplicado (window={w}).")
                elif method in ('linear','cubic'):
                    # Atualize o drop_duplicates para usar a coluna de tempo dinamicamente
                    time_col = next((col for col in dfi.columns if col.startswith('Time_s')), None)
                    if not time_col:
                        exibir_erro(f"Ensaio {idx+1}: Coluna de tempo não encontrada para interpolar.")
                        return
                    dfi.drop_duplicates(subset=[time_col], inplace=True)
                    if len(dfi) < 2:
                        exibir_erro(f"Ensaio {idx+1}: Após remover duplicatas, <2 pontos p/ interpolar.")
                        return
                    dfi.sort_values(time_col, inplace=True)
                    n_points = n_points_widget.value
                    kind = 'linear' if method == 'linear' else 'cubic'
                    f_dens = interp1d(dfi[time_col], dfi[dens_col], kind=kind, fill_value="extrapolate")
                    # Se houver coluna de temperatura, também interpolar
                    temp_col = next((col for col in dfi.columns if col.startswith('Temperature_C')), None)
                    f_temp = None
                    if temp_col:
                        f_temp = interp1d(dfi[time_col], dfi[temp_col], kind=kind, fill_value="extrapolate")

                    new_times = np.linspace(dfi[time_col].min(), dfi[time_col].max(), int(n_points))
                    df_new = pd.DataFrame({time_col: new_times})
                    df_new[dens_col] = f_dens(new_times)
                    if f_temp:
                        df_new[temp_col] = f_temp(new_times)

                    dfi = df_new
                    exibir_mensagem(f"Ensaio {idx+1}: Interpolação {kind} gerou {n_points} pontos.")
                else:
                    exibir_erro(f"Ensaio {idx+1}: Método de filtro desconhecido ({method}).")

                dfi.sort_values(time_col, inplace=True)
                dfi.reset_index(drop=True, inplace=True)
                self.dfs_ensaios[idx] = dfi

                exibir_mensagem(f"Ensaio {idx+1}: Filtro/Interp '{method}' finalizado. Tamanho DF={len(dfi)}.")
                self._update_slider_range(idx, slider_cut)
                # Salva estado após filtro/interpolação
                self.history.push(self.dfs_ensaios[idx], module_name=f"Module 3: Filtro/Interpolação Test {idx+1}")

            except Exception as e:
                exibir_erro(f"Ensaio {idx+1}: Erro ao filtrar/interpolar - {e}")


        btn_filter.on_click(on_apply_filter)

        # =============== Layout em duas colunas ===============
        panel_left = widgets.VBox([
            widgets.HTML(f"<b>Ajuste de Tempo (Ensaio {idx+1})</b>"),
            radio_zerar,
            btn_aplicar_zerar,

            widgets.HTML("<b>Corte de Intervalo</b>"),
            slider_cut,
            btn_cut,

            widgets.HTML("<b>Filtragem / Interpolação</b>"),
            filter_dropdown,
            bin_size_widget,
            n_points_widget,
            btn_filter,

            widgets.HTML("<b>Exibir Curvas / Exportar</b>"),
            widgets.HBox([btn_plot, btn_export]),

            vbox_out  # logs
        ])

        panel_right = widgets.VBox([fig_out])  # apenas a saída de figura

        panel = widgets.HBox([panel_left, panel_right])

        return panel

    def _on_filter_change(self, change, bin_size_widget, n_points_widget):
        """
        Ajusta exibição de bin_size_widget e n_points_widget conforme método de filtragem.
        """
        new_val = change['new']
        if new_val == 'orlandini':
            bin_size_widget.layout.display = ''
            n_points_widget.layout.display = 'none'
        elif new_val in ('linear','cubic'):
            bin_size_widget.layout.display = 'none'
            n_points_widget.layout.display = ''
        else:
            bin_size_widget.layout.display = 'none'
            n_points_widget.layout.display = 'none'

    def _get_time_range(self, idx):
        dfi = self.dfs_ensaios[idx]
        # Procura a coluna que inicia com "Time_s" (independente do sufixo)
        time_cols = [col for col in dfi.columns if col.startswith('Time_s')]
        if dfi is not None and not dfi.empty and time_cols:
            time_col = time_cols[0]  # assume que há apenas uma coluna de tempo por ensaio
            return float(dfi[time_col].min()), float(dfi[time_col].max())
        else:
            return 0.0, 1.0


    def _update_slider_range(self, idx, slider_cut):
        """
        Recalcula min/max do Time_s e redefine slider_cut.
        """
        tmin, tmax = self._get_time_range(idx)
        slider_cut.min = tmin
        slider_cut.max = tmax
        slider_cut.value = (tmin, tmax)

    def display(self):
        if hasattr(self, 'main_ui'):
            display(self.main_ui)
        else:
            display(widgets.HTML("<b>Nenhuma interface disponível (n_ensaios=0).</b>"))

    def get_dfs(self):
        return self.dfs_ensaios


def iniciar_modulo3(dfs_ensaios):
    """
    Função auxiliar para instanciar e exibir o Módulo 3.
    """
    mod3 = Modulo3Recorte(dfs_ensaios)
    return mod3


In [ ]:
# modulo_roteadormetodo.py
# Exemplo de célula final do seu notebook

import ipywidgets as widgets
from IPython.display import display, clear_output

# Texto de apresentação - somente será exibido agora
intro_text = """
<div style="font-family: Arial, sans-serif; line-height: 1.5;">
    <h2>Final Steps - Ogum Sintering</h2>
    <p>
        Obrigado por utilizar o Ogum Sintering até aqui.
        Agora você pode escolher entre os métodos finais
        para realizar a análise de dados. O Cálculo de LogTheta,
        de acordo com os trabalhos de Su e Johnson para Curvas mestras
        ou a Confecção de Curvas de Arhhenius:
    </p>
</div>
"""

# RadioButtons para escolha do método
method_radio = widgets.RadioButtons(
    options=["LogTheta ( Curvas Mestras )", "Curvas de Arrhenius "],
    value="LogTheta ( Curvas Mestras )",
    description="Method:",
    style={'description_width': 'initial'}
)


# Botão para prosseguir
proceed_button = widgets.Button(description="Realizar Ação", button_style='success')

# Área de saída para logs
out = widgets.Output()

def on_proceed_clicked(b):
    with out:
        clear_output()
        choice = method_radio.value
        if choice == "LogTheta ( Curvas Mestras )":
            print("[INFO] User chose LogTheta method.")
            # Chame aqui o(s) módulo(s) correspondentes ao LogTheta
            # Por exemplo: iniciar_modulo_logtheta()
        else:
            print("[INFO] User chose Arrhenius method.")
            # Chame aqui o(s) módulo(s) correspondentes ao Arrhenius
            # Por exemplo: iniciar_modulo_arrhenius()
    # Opcional: Oculta a interface de escolha após prosseguir
    ui.layout.display = 'none'


proceed_button.on_click(on_proceed_clicked)

# Monta a interface final
ui = widgets.VBox([
    widgets.HTML(value=intro_text),
    method_radio,
    proceed_button,
    out
])




In [ ]:
# modulo4_logtheta.py (VERSÃO FINAL SIMPLIFICADA)

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List

# Importações necessárias
# from core import SinteringDataRecord
# from modulo1_interface import EaSelectionWidget, exibir_mensagem, exibir_erro, ...

class ModuloLogTheta:
    """
    Classe refatorada e simplificada para o cálculo de log(theta).
    Usa o widget reutilizável EaSelectionWidget para a definição das energias.
    """
    def __init__(self, dfs_ensaios):
        self.dfs_ensaios = [df for df in dfs_ensaios if df is not None and not df.empty]
        self.n_ensaios = len(self.dfs_ensaios)
        self.sintering_records: List[SinteringDataRecord] = []

        # 1. Instancia o novo widget reutilizável
        self.ea_selector = EaSelectionWidget()

        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        # 2. A UI deste módulo agora é muito mais limpa
        self.btn_calc_logtheta = widgets.Button(description="Calcular LogTheta", button_style='success')
        self.btn_calc_logtheta.on_click(self._on_calc_logtheta)

        self.btn_plot_curves = widgets.Button(description="Visualizar Curvas", button_style='warning')
        self.btn_plot_curves.on_click(self._on_plot_curves)

        self.btn_export_logtheta = widgets.Button(description="Exportar Resultados", button_style='info')
        self.btn_export_logtheta.on_click(self._on_export_logtheta)

        self.ui = widgets.VBox([
            widgets.HTML("<h3>Módulo 4: Cálculo de LogTheta</h3>"),
            # 3. Apenas exibe a UI do widget seletor de Ea
            self.ea_selector.ui,
            widgets.HBox([self.btn_calc_logtheta, self.btn_plot_curves, self.btn_export_logtheta]),
            self.out
        ])

    def _on_calc_logtheta(self, b):
        with self.out:
            clear_output()

        self.sintering_records = []

        # 4. Pega a lista de energias diretamente do widget
        eas = self.ea_selector.get_ea_list()

        if not eas:
            # A mensagem de erro já é exibida pelo próprio widget
            return

        # A lógica de cálculo daqui para baixo permanece EXATAMENTE A MESMA.
        # Nenhuma alteração é necessária no loop de cálculo.
        exibir_mensagem(f"Iniciando cálculo... Processando {len(eas)} energias de ativação.")

        for i, df_ensaio in enumerate(self.dfs_ensaios):
            time_col = next((col for col in df_ensaio.columns if col.startswith('Time_s')), None)
            temp_col = next((col for col in df_ensaio.columns if col.startswith('Temperature_C')), None)
            dens_col = next((col for col in df_ensaio.columns if col.startswith('DensidadePct')), None)

            if not (time_col and temp_col and dens_col):
                exibir_erro(f"Ensaio {i+1}: Colunas essenciais não encontradas. Pulando.")
                continue

            if df_ensaio[time_col].size < 2:
                exibir_erro(f"Ensaio {i+1}: Dados insuficientes para integração. Pulando.")
                continue

            T_k = df_ensaio[temp_col].values + 273.15

            for Ea_kj in eas:
                try:
                    Ea_j = Ea_kj * 1000.0
                    theta_inst = (1.0 / T_k) * np.exp(-Ea_j / (R * T_k))
                    integrated = cumtrapz(theta_inst, df_ensaio[time_col].values, initial=0)

                    with np.errstate(divide='ignore', invalid='ignore'):
                        log_integrated = np.log10(integrated)

                    log_integrated[~np.isfinite(log_integrated)] = np.nan

                    df_record = pd.DataFrame({
                        'logtheta': log_integrated,
                        'valor': df_ensaio[dens_col].values
                    })

                    record = SinteringDataRecord(
                        ensaio_id=i, Ea=Ea_kj,
                        tipo_dado_y='densidade_original', df=df_record
                    )
                    self.sintering_records.append(record)

                except Exception as e:
                    exibir_erro(f"Ensaio {i+1}: Falha ao calcular para Ea={Ea_kj:.2f}. Erro: {e}")

        exibir_mensagem(f"Cálculo concluído. {len(self.sintering_records)} SinteringDataRecord's foram gerados.")


    def _on_plot_curves(self, b):
        """
        ALTERADO: A plotagem agora itera sobre os SinteringDataRecords.
        """
        with self.out:
            clear_output()

        if not self.sintering_records:
            exibir_erro("Nenhum dado calculado para plotar. Por favor, execute o cálculo primeiro.")
            return

        # Agrupa os records por ensaio_id para plotar um gráfico por ensaio
        from collections import defaultdict
        records_por_ensaio = defaultdict(list)
        for rec in self.sintering_records:
            records_por_ensaio[rec.ensaio_id].append(rec)

        for ensaio_id, records in records_por_ensaio.items():
            fig, ax = plt.subplots(figsize=(8, 6))
            ax.set_title(f"Ensaio {ensaio_id + 1} – Densidade vs. LogTheta")
            ax.set_xlabel("log(theta)")
            ax.set_ylabel("Densidade (%)")

            for rec in records:
                df_plot = rec.df.dropna() # Remove pontos onde logtheta ou valor são NaN
                if len(df_plot) < 2:
                    continue

                label = f"Ea={rec.Ea:.2f} kJ/mol"
                ax.plot(df_plot['logtheta'], df_plot['valor'], 'o-', ms=3, label=label)

            ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            ax.grid(True)
            plt.tight_layout()
            plt.show()

        exibir_mensagem("Visualização concluída.")

# Em ModuloLogTheta

    def _on_export_logtheta(self, b):
        """
        ALTERADO: Exportação agora cria um único DataFrame em formato "largo" (horizontal),
        com colunas nomeadas dinamicamente.
        """
        with self.out:
            clear_output()

        if not self.sintering_records:
            exibir_erro("Nenhum dado disponível para exportação. Execute o cálculo primeiro.")
            return

        all_dfs_to_export = []
        for rec in self.sintering_records:
            # Cria um nome de coluna base para este record
            base_name = f"E{rec.ensaio_id + 1}_Ea{rec.Ea:.2f}"

            # Renomeia as colunas do df interno para o formato de exportação
            df_renamed = rec.df.copy()
            df_renamed.columns = [f"logtheta_{base_name}", f"densidade_{base_name}"]
            all_dfs_to_export.append(df_renamed)

        # Concatena todos os DataFrames horizontalmente
        if not all_dfs_to_export:
            exibir_erro("Nenhum DataFrame válido para concatenar.")
            return

        df_export = pd.concat(all_dfs_to_export, axis=1)

        link = gerar_link_download(df_export, nome_arquivo="Resultados_LogTheta_Largo.xlsx")
        display(link)
        exibir_mensagem("Exportação em formato 'Largo' (horizontal) concluída.")

    def get_sintering_records(self) -> List[SinteringDataRecord]:
        """
        NOVO: Método para passar a lista de records para o próximo módulo.
        """
        return self.sintering_records

In [ ]:
# modulo5_1_alinhamento.py (VERSÃO REFATORADA)

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from collections import defaultdict
from typing import List

# Importa a estrutura de dados central. Garanta que a definição da classe
# SinteringDataRecord esteja acessível.
# from core import SinteringDataRecord

# Funções utilitárias (exibir_mensagem, exibir_erro, etc.)
# ...

class Modulo5_1Alinhamento:
    """
    Módulo refatorado para alinhar curvas de sinterização.
    Recebe uma lista de SinteringDataRecord e produz uma nova lista
    com os dados alinhados em um eixo comum (logtheta ou densidade).
    """
    def __init__(self, sintering_records: List[SinteringDataRecord]):
        # Filtra apenas os records de entrada relevantes ('densidade_original')
        self.input_records = [rec for rec in sintering_records if rec.tipo_dado_y == 'densidade_original']

        # Armazena todos os records (incluindo os que não serão processados) para passar adiante
        self.all_input_records = sintering_records

        # Lista para armazenar os novos records com dados alinhados
        self.aligned_records: List[SinteringDataRecord] = []

        self.out = widgets.Output()
        self._build_ui()

# ========================================================================
# MÉTODO CORRIGIDO para a classe Modulo5_1Alinhamento
# ========================================================================

    def _build_ui(self):
        if not self.input_records:
            self.main_ui = widgets.HTML("<b>Nenhum dado de 'densidade_original' disponível para alinhamento.</b>")
            return

        # --- AJUSTE DE LAYOUT APLICADO AQUI ---

        # 1. Cria um título HTML separado para a descrição
        axis_label = widgets.HTML("<b>Eixo de Alinhamento:</b>")

        # 2. Usa textos mais curtos nas opções do RadioButtons
        self.radio_axis = widgets.RadioButtons(
            options=[
                ("Eixo comum de log(θ)", "logtheta"),
                ("Eixo comum de Densidade", "dens")
            ],
            value="dens",
            # A propriedade 'description' foi removida para evitar sobreposição
        )

        self.int_n_points = widgets.IntText(
            value=200,
            description="Nº de Pontos:",
            tooltip="Número de pontos para a nova grade de interpolação."
        )

        self.btn_apply = widgets.Button(description="Aplicar Alinhamento", button_style='success')
        self.btn_apply.on_click(self._on_apply_alignment)

        # 3. Monta o layout principal com o novo título e os botões
        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.1 – Alinhamento e Interpolação</h3>"),
            widgets.HTML("<p>Selecione o eixo para criar uma grade comum e clique em 'Aplicar'. As curvas serão reamostradas nesta grade.</p>"),
            axis_label, # Adiciona o título separado
            self.radio_axis, # Adiciona os RadioButtons
            self.int_n_points,
            self.btn_apply,
            self.out
        ])

    def display(self):
        display(self.main_ui)

    def _on_apply_alignment(self, b):
        with self.out:
            clear_output()
            exibir_mensagem("Iniciando processo de alinhamento...")

        self.aligned_records = []  # Limpa resultados de execuções anteriores
        mode = self.radio_axis.value
        n_points = self.int_n_points.value

        if n_points < 2:
            exibir_erro("Número de pontos deve ser no mínimo 2.")
            return

        # 1. Agrupar records por Energia de Ativação
        records_by_ea = defaultdict(list)
        for rec in self.input_records:
            records_by_ea[rec.Ea].append(rec)

        # 2. Processar cada grupo de Ea
        for Ea, records_group in records_by_ea.items():
            # Itera sobre cada record no grupo para criar um novo record alinhado
            for rec_to_process in records_group:
                df = rec_to_process.df.dropna().copy()
                if len(df) < 2:
                    exibir_erro(f"Ensaio {rec_to_process.ensaio_id+1}, Ea={Ea}: Pontos insuficientes (<2) após remover NaNs. Pulando.")
                    continue

                try:
                    if mode == "logtheta":
                        # Alinhar em um eixo comum de log(theta)
                        df.sort_values('logtheta', inplace=True)
                        x_axis = 'logtheta'
                        y_axis = 'valor'
                        x_label_aligned = 'logtheta_alinhado'
                        y_label_aligned = 'densidade_alinhada'

                    elif mode == "dens":
                        # Alinhar em um eixo comum de densidade
                        df.sort_values('valor', inplace=True)
                        x_axis = 'valor'
                        y_axis = 'logtheta'
                        x_label_aligned = 'densidade_alinhada'
                        y_label_aligned = 'logtheta_alinhado'

                    else:
                        continue # Modo inválido

                    # Determina a faixa para a nova grade
                    x_min, x_max = df[x_axis].min(), df[x_axis].max()
                    if np.isclose(x_min, x_max):
                        exibir_erro(f"Ensaio {rec_to_process.ensaio_id+1}, Ea={Ea}: Eixo '{x_axis}' tem valores constantes. Pulando alinhamento.")
                        continue

                    # Cria a nova grade e o interpolador
                    new_x_grid = np.linspace(x_min, x_max, n_points)
                    interpolator = interp1d(df[x_axis], df[y_axis], kind='linear', bounds_error=False, fill_value=np.nan)
                    new_y_values = interpolator(new_x_grid)

                    # Cria o novo DataFrame com dados alinhados e nomes de coluna padronizados
                    df_aligned_result = pd.DataFrame({
                        x_label_aligned: new_x_grid,
                        y_label_aligned: new_y_values
                    }).dropna()

                    if df_aligned_result.empty:
                        exibir_erro(f"Ensaio {rec_to_process.ensaio_id+1}, Ea={Ea}: Interpolação resultou em DataFrame vazio. Verifique os dados de entrada.")
                        continue

                    # Cria o novo SinteringDataRecord com o resultado
                    aligned_rec = SinteringDataRecord(
                        ensaio_id=rec_to_process.ensaio_id,
                        Ea=Ea,
                        tipo_dado_y='dados_alinhados', # Novo tipo de dado padronizado
                        df=df_aligned_result,
                        metadata={'alignment_axis': mode} # Guarda metadados sobre o processo
                    )
                    self.aligned_records.append(aligned_rec)

                except Exception as e:
                    exibir_erro(f"Falha ao alinhar Ensaio {rec_to_process.ensaio_id+1} para Ea={Ea}. Erro: {e}")

        exibir_mensagem(f"Alinhamento concluído. {len(self.aligned_records)} records de 'dados_alinhados' foram gerados.")

    def get_aligned_records(self) -> List[SinteringDataRecord]:
        """
        Retorna a lista de records alinhados mais os records originais de entrada.
        Isso garante que os próximos módulos tenham acesso a todo o histórico de dados.
        """
        return self.all_input_records + self.aligned_records

In [ ]:
# modulo5_2_blaine.py (VERSÃO REFATORADA)

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
from typing import List

# Importa a estrutura de dados central
# from core import SinteringDataRecord

# Funções utilitárias (exibir_mensagem, exibir_erro, gerar_link_download, etc.)
# ...

class Modulo5_2BlaineParameters:
    """
    Submódulo 5.2 Refatorado – Cálculo de Parâmetros de Densificação (Psi e Phi).

    Este módulo consome records do tipo 'dados_alinhados' e produz novos
    records do tipo 'psi' e 'phi'.
    """
    def __init__(self, sintering_records: List[SinteringDataRecord]):
        # Armazena todos os records de entrada
        self.input_records = sintering_records

        # Filtra os records que este módulo irá processar
        self.records_to_process = [rec for rec in sintering_records if rec.tipo_dado_y == 'dados_alinhados']

        # Lista para armazenar os novos records gerados ('psi' e 'phi')
        self.processed_records: List[SinteringDataRecord] = []

        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        if not self.records_to_process:
            self.main_ui = widgets.HTML("<b>Nenhum dado do tipo 'dados_alinhados' encontrado para processar.</b>")
            return

        # A interface foi simplificada. A seleção de 'Ea' não é mais necessária aqui.
        self.float_rho0 = widgets.FloatText(
            value=55.0, # Um valor inicial comum
            description="ρ0 (Dens. Inicial %):",
            tooltip="Densidade inicial da peça verde em porcentagem (ex: 55.0)."
        )

        self.btn_calc = widgets.Button(description="Calcular Parâmetros (Psi, Phi)", button_style='success')
        self.btn_calc.on_click(self._on_calc_blaine)

        self.btn_plot = widgets.Button(description="Visualizar Curvas (Psi, Phi)", button_style='warning')
        self.btn_plot.on_click(self._on_plot)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.2 – Parâmetros de Densificação (Blaine)</h3>"),
            widgets.HTML("<p>Defina a densidade inicial (ρ0) e clique em 'Calcular' para gerar os parâmetros ψ e φ.</p>"),
            self.float_rho0,
            widgets.HBox([self.btn_calc, self.btn_plot]),
            self.out
        ])

    def display(self):
        display(self.main_ui)

    def _on_calc_blaine(self, b):
        with self.out:
            clear_output()
            exibir_mensagem("Iniciando cálculo dos parâmetros de densificação...")

        rho0 = self.float_rho0.value
        if not (0 < rho0 < 100):
            exibir_erro("O valor de ρ0 (Densidade Inicial) deve estar entre 0 e 100.")
            return

        self.processed_records = [] # Limpa resultados anteriores

        for rec in self.records_to_process:
            try:
                df_input = rec.df
                # Os dados de densidade estão na coluna 'densidade_alinhada'
                dens_values = df_input['densidade_alinhada'].values
                # O eixo X correspondente é 'logtheta_alinhado'
                logtheta_values = df_input['logtheta_alinhado'].values

                # --- Cálculo de Psi (ψ) ---
                # ψ = (ρ - ρ₀) / (100 - ρ₀)
                psi_vals = (dens_values - rho0) / (100.0 - rho0)
                df_psi = pd.DataFrame({'logtheta_alinhado': logtheta_values, 'valor': psi_vals})

                rec_psi = SinteringDataRecord(
                    ensaio_id=rec.ensaio_id,
                    Ea=rec.Ea,
                    tipo_dado_y='psi',
                    df=df_psi,
                    metadata=rec.metadata.copy() # Propaga metadados anteriores
                )
                self.processed_records.append(rec_psi)

                # --- Cálculo de Phi (Φ) ---
                # Φ = (ρ - ρ₀) / (100 - ρ)
                # Adicionar uma pequena constante ao denominador para evitar divisão por zero
                phi_vals = (dens_values - rho0) / (100.0 - dens_values + 1e-9)
                df_phi = pd.DataFrame({'logtheta_alinhado': logtheta_values, 'valor': phi_vals})

                rec_phi = SinteringDataRecord(
                    ensaio_id=rec.ensaio_id,
                    Ea=rec.Ea,
                    tipo_dado_y='phi',
                    df=df_phi,
                    metadata=rec.metadata.copy()
                )
                self.processed_records.append(rec_phi)

            except KeyError as e:
                exibir_erro(f"Erro de chave ao processar record para Ea={rec.Ea}, Ensaio={rec.ensaio_id+1}. Coluna esperada não encontrada: {e}")
            except Exception as e:
                exibir_erro(f"Erro inesperado ao processar record para Ea={rec.Ea}, Ensaio={rec.ensaio_id+1}. Erro: {e}")

        exibir_mensagem(f"Cálculo concluído. {len(self.processed_records)} novos records ('psi' e 'phi') foram gerados.")

    def _on_plot(self, b):
        with self.out:
            clear_output()

        if not self.processed_records:
            exibir_erro("Nenhum parâmetro calculado para plotar. Execute o cálculo primeiro.")
            return

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=False)
        fig.suptitle("Parâmetros de Densificação vs. log(θ) Alinhado")

        # Plot para Psi
        ax1.set_title("Parâmetro ψ (Psi)")
        ax1.set_xlabel("log(theta) alinhado")
        ax1.set_ylabel("ψ = (ρ-ρ₀)/(1-ρ₀)")

        # Plot para Phi
        ax2.set_title("Parâmetro φ (Phi)")
        ax2.set_xlabel("log(theta) alinhado")
        ax2.set_ylabel("φ = (ρ-ρ₀)/(1-ρ)")

        for rec in self.processed_records:
            df_plot = rec.df.dropna()
            if df_plot.empty: continue

            label = f"E{rec.ensaio_id+1} Ea={rec.Ea:.1f}"

            if rec.tipo_dado_y == 'psi':
                ax1.plot(df_plot['logtheta_alinhado'], df_plot['valor'], 'o-', ms=3, label=label)
            elif rec.tipo_dado_y == 'phi':
                ax2.plot(df_plot['logtheta_alinhado'], df_plot['valor'], 'o-', ms=3, label=label)

        ax1.grid(True)
        ax1.legend()
        ax2.grid(True)
        ax2.legend()
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()


    def get_blaine_records(self) -> List[SinteringDataRecord]:
        """
        Retorna a lista completa de records: os originais de entrada mais os
        novos records 'psi' e 'phi' que foram processados.
        """
        return self.input_records + self.processed_records

In [ ]:
# modulo5_3_sigmoides.py (VERSÃO REFATORADA)

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from typing import List

# Importa a estrutura de dados central e a função de ajuste
# from core import SinteringDataRecord

# --- Definição da Função Sigmoidal ---
# (Pode ficar aqui ou em um módulo de utilidades/modelos)
def boltzmann_sigmoid(x, A1, A2, x0, dx):
    """ Boltzmann (logística básica): y = A2 + (A1 - A2) / (1 + exp((x - x0)/dx)) """
    # Salvaguarda contra overflow no exp para valores muito grandes/pequenos de x
    exp_term = np.exp(np.clip((x - x0) / dx, -700, 700))
    return A2 + (A1 - A2) / (1 + exp_term)

class Modulo5_3Sigmoides:
    """
    Submódulo 5.3 Refatorado – Ajuste Sigmoidal (Fitting)

    Consome SinteringDataRecords e gera novos records contendo os resultados
    do ajuste da curva de Boltzmann.
    """
    def __init__(self, sintering_records: List[SinteringDataRecord]):
        self.input_records = sintering_records
        # Lista para armazenar os novos records gerados com os ajustes
        self.processed_records: List[SinteringDataRecord] = []
        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        # Mapeamento da UI para os tipos de dados internos
        self.DATA_OPTIONS_MAP = {
            "Densidade Alinhada": "dados_alinhados",
            "Parâmetro Psi (ψ)": "psi",
            "Parâmetro Phi (φ)": "phi",
            "Densidade Original": "densidade_original"
        }

        self.select_data = widgets.SelectMultiple(
            options=list(self.DATA_OPTIONS_MAP.keys()),
            description="Ajustar dados de:",
            rows=4
        )

        self.btn_fit = widgets.Button(description="Aplicar Ajuste Boltzmann", button_style='success')
        self.btn_fit.on_click(self._on_fit_boltzmann)

        self.btn_plot = widgets.Button(description="Visualizar Ajustes", button_style='warning')
        self.btn_plot.on_click(self._on_plot)

        self.btn_export = widgets.Button(description="Exportar Ajustes", button_style='info', disabled=True)
        self.btn_export.on_click(self._on_export)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.3 – Ajuste Sigmoidal (Fitting)</h3>"),
            widgets.HTML("<p>Selecione um ou mais conjuntos de dados para aplicar o ajuste com a função de Boltzmann.</p>"),
            self.select_data,
            widgets.HBox([self.btn_fit, self.btn_plot]),
            self.out
        ])

    def display(self):
        display(self.main_ui)

# Em Modulo5_3Sigmoides

    def _on_fit_boltzmann(self, b):
        with self.out:
            clear_output()
            exibir_mensagem("Iniciando ajuste sigmoidal de Boltzmann...")

        selected_options = list(self.select_data.value)
        if not selected_options:
            exibir_erro("Nenhum conjunto de dados selecionado para o ajuste.")
            return

        tipos_dado_para_ajustar = [self.DATA_OPTIONS_MAP[opt] for opt in selected_options]
        self.processed_records = []
        records_to_process = [rec for rec in self.input_records if rec.tipo_dado_y in tipos_dado_para_ajustar]

        if not records_to_process:
            exibir_erro("Nenhum dado correspondente à sua seleção foi encontrado.")
            return

        for rec in records_to_process:
            try:
                df_clean = rec.df.dropna()
                if len(df_clean) < 4: continue

                # --- LÓGICA CORRIGIDA PARA ENCONTRAR COLUNAS X E Y ---
                x_col = next((col for col in df_clean.columns if 'logtheta' in col), None)
                # O que não for a coluna X, será a Y.
                y_col = next((col for col in df_clean.columns if col != x_col), None)

                if x_col is None or y_col is None:
                    exibir_erro(f"Não foi possível identificar as colunas X e Y no record {rec.tipo_dado_y} (Ensaio {rec.ensaio_id+1}, Ea={rec.Ea}).")
                    continue

                x_data = df_clean[x_col].values
                y_data = df_clean[y_col].values
                # --- FIM DA CORREÇÃO ---

                p0 = [np.max(y_data), np.min(y_data), np.median(x_data), (np.max(x_data) - np.min(x_data)) / 4]
                popt, _ = curve_fit(boltzmann_sigmoid, x_data, y_data, p0=p0, maxfev=10000)
                y_fit_values = boltzmann_sigmoid(x_data, *popt)
                df_fit_result = pd.DataFrame({x_col: x_data, 'valor_ajustado': y_fit_values})

                fit_record = SinteringDataRecord(
                    ensaio_id=rec.ensaio_id, Ea=rec.Ea,
                    tipo_dado_y=f"fit_boltzmann_{rec.tipo_dado_y}",
                    df=df_fit_result,
                    metadata={'popt': popt.tolist(), 'source_type': rec.tipo_dado_y}
                )
                self.processed_records.append(fit_record)

            except Exception as e:
                # O erro KeyError('valor') não deve mais ocorrer com a lógica acima
                exibir_erro(f"Erro inesperado durante o ajuste do record {rec.tipo_dado_y} (Ensaio {rec.ensaio_id+1}, Ea={rec.Ea}): {e}")

        exibir_mensagem(f"Ajuste concluído. {len(self.processed_records)} novos records de 'fit_boltzmann_*' foram gerados.")
        # Habilita o botão se houver resultados para exportar
        if self.processed_records:
                self.btn_export.disabled = False

# Em Modulo5_3Sigmoides

    def _on_plot(self, b):
        with self.out:
            clear_output()

        if not self.processed_records:
            exibir_erro("Nenhum ajuste realizado para plotar. Execute o ajuste primeiro.")
            return

        for fit_rec in self.processed_records:
            # Encontra o record original que deu origem a este ajuste
            source_rec = next((rec for rec in self.input_records
                               if rec.ensaio_id == fit_rec.ensaio_id
                               and rec.Ea == fit_rec.Ea
                               and rec.tipo_dado_y == fit_rec.metadata.get('source_type')), None)

            if source_rec is None: continue

            fig, ax = plt.subplots(figsize=(8, 6))
            title = f"Ajuste Boltzmann - {fit_rec.metadata.get('source_type')}\n(Ensaio {fit_rec.ensaio_id+1}, Ea={fit_rec.Ea:.2f} kJ/mol)"
            ax.set_title(title)

            # --- LÓGICA CORRIGIDA PARA ENCONTRAR COLUNAS X E Y DINAMICAMENTE ---
            df_orig_plot = source_rec.df.dropna()

            x_col_orig = next((col for col in df_orig_plot.columns if 'logtheta' in col), None)
            y_col_orig = next((col for col in df_orig_plot.columns if col != x_col_orig), None)

            if x_col_orig is None or y_col_orig is None:
                exibir_erro(f"Não foi possível plotar o record original para {fit_rec.tipo_dado_y}")
                continue

            ax.set_xlabel(x_col_orig)
            ax.set_ylabel(source_rec.tipo_dado_y.replace('_', ' ').title())

            # Plota dados originais usando as colunas dinâmicas
            ax.plot(df_orig_plot[x_col_orig], df_orig_plot[y_col_orig], 'o', label="Dados Originais", markersize=5, alpha=0.7)

            # Plota curva ajustada
            x_col_fit = next((col for col in fit_rec.df.columns if 'logtheta' in col), None)
            df_fit_plot = fit_rec.df.sort_values(x_col_fit)
            ax.plot(df_fit_plot[x_col_fit], df_fit_plot['valor_ajustado'], '-', color='red', linewidth=2, label="Ajuste Boltzmann")

            ax.legend()
            ax.grid(True)
            plt.tight_layout()
            plt.show()

# ========================================================================
# ADICIONE ESTE MÉTODO COMPLETO DENTRO DA CLASSE Modulo5_3Sigmoides
# ========================================================================

    def _on_export(self, b):
        with self.out:
            clear_output()

        if not self.processed_records:
            exibir_erro("Nenhum resultado de ajuste disponível para exportar. Execute o ajuste primeiro.")
            return

        exibir_mensagem("Preparando dados para exportação...")

        lista_de_dfs_para_export = []

        # Itera sobre cada resultado de ajuste
        for fit_rec in self.processed_records:
            # Encontra o record com os dados originais que correspondem a este ajuste
            source_rec = next((rec for rec in self.input_records
                               if rec.ensaio_id == fit_rec.ensaio_id
                               and rec.Ea == fit_rec.Ea
                               and rec.tipo_dado_y == fit_rec.metadata.get('source_type')), None)

            if source_rec is None:
                continue

            # Junta os dados originais e os dados ajustados
            x_col = next((col for col in source_rec.df.columns if 'logtheta' in col), 'logtheta')
            y_col = next((col for col in source_rec.df.columns if col != x_col), 'valor')

            # Renomeia colunas para clareza
            df_source = source_rec.df.rename(columns={x_col: 'logtheta', y_col: 'valor_original'})
            df_fit = fit_rec.df.rename(columns={x_col: 'logtheta', 'valor_ajustado': 'valor_ajustado'})

            # Une os dois dataframes usando 'logtheta' como chave
            df_merged = pd.merge(df_source, df_fit, on='logtheta', how='outer')

            # Adiciona os metadados
            df_merged['Ensaio_ID'] = fit_rec.ensaio_id + 1
            df_merged['Ea_kJ_mol'] = fit_rec.Ea
            df_merged['Tipo_Dado'] = fit_rec.metadata.get('source_type')

            # Adiciona os parâmetros do ajuste (popt)
            popt = fit_rec.metadata.get('popt', [np.nan] * 4)
            df_merged['popt_A1'] = popt[0]
            df_merged['popt_A2'] = popt[1]
            df_merged['popt_x0'] = popt[2]
            df_merged['popt_dx'] = popt[3]

            lista_de_dfs_para_export.append(df_merged)

        if not lista_de_dfs_para_export:
            exibir_erro("Não foi possível gerar dados para exportação.")
            return

        # Concatena tudo em um único DataFrame final
        df_export_final = pd.concat(lista_de_dfs_para_export, ignore_index=True)

        # Reorganiza as colunas para melhor leitura
        col_order = [
            'Ensaio_ID', 'Ea_kJ_mol', 'Tipo_Dado', 'logtheta', 'valor_original', 'valor_ajustado',
            'popt_A1', 'popt_A2', 'popt_x0', 'popt_dx'
        ]
        df_export_final = df_export_final[col_order]

        # Gera o link para download
        link = gerar_link_download(df_export_final, nome_arquivo="Resultados_Ajuste_Boltzmann.xlsx")
        display(link)
        exibir_mensagem("Exportação dos resultados do ajuste concluída.")

    def get_sigmoidal_records(self) -> List[SinteringDataRecord]:
        """
        Retorna a lista completa de records: os de entrada mais os novos
        records com os resultados dos ajustes.
        """
        return self.input_records + self.processed_records

In [ ]:
# modulo5_4_roteador.py (VERSÃO FINAL CORRIGIDA)

import ipywidgets as widgets
from IPython.display import display, clear_output
from typing import List

# As importações abaixo são necessárias para o funcionamento.
# Elas devem estar definidas em células anteriores do notebook.
# from core import SinteringDataRecord
# from modulo5_4_1_comparaçao import Modulo5_4_1Comparisons
# from modulo5_4_2_Ref import Modulo5_4_2Ref
# from modulo5_4_3_blaine_linear import Modulo5_4_3BlaineLinear
# from modulo1_interface import exibir_mensagem, exibir_erro

class Modulo5_4_Roteador:
    """
    Módulo 5.4.0 Refatorado – Roteador para os métodos de análise final.

    Apresenta ao usuário as opções de análise final (Comparação, Erro Individual,
    Linearização) e, com base na escolha, instancia e exibe o submódulo correspondente,
    passando a lista completa de SinteringDataRecord.
    """

    def __init__(self, sintering_records: List[SinteringDataRecord], controller=None):
        """
        Construtor ajustado para receber a lista de SinteringDataRecord.

        :param sintering_records: A lista completa de dados processados até esta etapa.
        :param controller: Referência ao MainInteractive para navegação.
        """
        # CORREÇÃO: Garante que o nome da variável está correto.
        self.sintering_records = sintering_records
        self.controller = controller
        self.out = widgets.Output()
        self._build_ui()

# Em Modulo5_4_Roteador

    def _build_ui(self):
        # A definição de 'explanation_text' e 'radio_paths' continua a mesma...
        explanation_text = """...""" # Seu texto de explicação
        self.html_explanation = widgets.HTML(value=explanation_text)

        self.radio_paths = widgets.RadioButtons(
            options=[
                ("1. Comparação de Curvas (MRS/MPCD)", "comparisons"),
                ("2. Erro de Ajuste Individual (MSE)", "individual_error"),
                ("3. Linearização de Blaine", "blaine_linear")
            ],
            value="comparisons",
            # A description longa pode ser removida e colocada em um HTML separado
        )

        # Botões
        self.btn_proceed = widgets.Button(description="Executar Análise", button_style='success')
        self.btn_proceed.on_click(self._on_proceed_clicked)
        self.btn_back = widgets.Button(description="Voltar (p/ Módulo 5.3)", button_style='warning')
        self.btn_back.on_click(self._on_back_clicked)

        # --- SOLUÇÃO PARA O LAYOUT ---
        # Colocamos o RadioButtons dentro de uma VBox para garantir a disposição vertical.
        self.main_ui = widgets.VBox([
            self.html_explanation,
            widgets.HTML("<b>Escolha o método de análise final:</b>"),
            widgets.VBox([self.radio_paths]), # <-- MUDANÇA AQUI
            widgets.HBox([self.btn_back, self.btn_proceed]),
            self.out
        ])

    def display(self):
        """Exibe a interface principal do roteador."""
        clear_output(wait=True)
        display(self.main_ui)

    def _on_proceed_clicked(self, b):
        """
        Callback para o botão 'Executar Análise'.
        Instancia o submódulo escolhido, passando a lista de SinteringDataRecord.
        """
        with self.out:
            clear_output()

        choice = self.radio_paths.value

        # CORREÇÃO: Passa 'self.sintering_records' para os construtores dos submódulos
        if choice == "comparisons":
            mod = Modulo5_4_1Comparisons(self.sintering_records, parent=self)
            mod.display()
        elif choice == "individual_error":
            mod = Modulo5_4_2Ref(self.sintering_records, parent=self)
            mod.display()
        elif choice == "blaine_linear":
            mod = Modulo5_4_3BlaineLinear(self.sintering_records, parent=self)
            mod.display()

    def _on_back_clicked(self, b):
        """
        Callback para o botão 'Voltar'.
        Chama um método no controlador principal para navegar para a tela anterior.
        """
        with self.out:
            clear_output()

        if self.controller and hasattr(self.controller, 'show_module_by_step'):
            exibir_mensagem("Retornando ao Módulo 5.3 (Ajuste Sigmoidal)...")
            # O passo 7 no MainInteractive corresponde ao Módulo 5.3
            self.controller.show_module_by_step(7)
        else:
            exibir_erro("Controlador ou método de retorno não foi encontrado.")

In [ ]:
# modulo5_4_1_comparaçao.py (VERSÃO REFATORADA)

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.optimize import minimize_scalar
from typing import List
from collections import defaultdict

# Importa a estrutura de dados central
# from core import SinteringDataRecord

# Funções utilitárias (exibir_mensagem, exibir_erro, gerar_link_download, etc.)
# ...

class Modulo5_4_1Comparisons:
    """
    Submódulo 5.4.1 Refatorado – Comparações (MRS e MPCD)

    Consome uma lista de SinteringDataRecord e usa as métricas MRS ou MPCD
    para encontrar a Energia de Ativação ótima que melhor sobrepõe as curvas.
    """

    def __init__(self, sintering_records: List[SinteringDataRecord], parent=None):
        self.all_records = sintering_records
        self.parent = parent
        self.results_df = None
        self.comparison_details_df = None
        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        # Mapeamento da UI para os tipos de dados internos
        self.DATA_OPTIONS_MAP = {
            "Densidade Alinhada": "dados_alinhados",
            "Parâmetro Psi (ψ)": "psi",
            "Parâmetro Phi (φ)": "phi",
            "Fit Boltzmann (Dens. Alinhada)": "fit_boltzmann_dados_alinhados",
            # Adicione outros mapeamentos conforme necessário
        }

        # Filtra opções para mostrar apenas os dados disponíveis
        available_types = set(rec.tipo_dado_y for rec in self.all_records)
        self.available_options = [ui_text for ui_text, internal_type in self.DATA_OPTIONS_MAP.items() if internal_type in available_types]

        if not self.available_options:
            self.main_ui = widgets.HTML("<b>Nenhum tipo de dado compatível com a comparação foi encontrado.</b>")
            return

        self.select_data = widgets.SelectMultiple(
            options=self.available_options,
            description="Comparar dados de:",
            rows=min(len(self.available_options), 6)
        )

        self.radio_method = widgets.RadioButtons(
            options=[("Mean Residual Square (MRS)", "mrs"), ("Mean Perpendicular Curve Distance (MPCD)", "mpcd")],
            value="mrs", description="Método:"
        )

        self.btn_compare = widgets.Button(description="Executar Comparação", button_style='success')
        self.btn_compare.on_click(self._on_compare)

        self.btn_export = widgets.Button(description="Exportar Resultados", button_style='info', disabled=True)
        self.btn_export.on_click(self._on_export)

        self.btn_back = widgets.Button(description="Voltar", button_style='warning')
        self.btn_back.on_click(self._on_back_clicked)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.4.1 – Comparação para Melhor Energia de Ativação</h3>"),
            widgets.HTML("<p>Selecione os conjuntos de dados e o método para encontrar a Ea que melhor sobrepõe as curvas.</p>"),
            self.select_data,
            self.radio_method,
            widgets.HBox([self.btn_back, self.btn_compare, self.btn_export]),
            self.out
        ])

    def display(self):
        display(self.main_ui)

    def _on_compare(self, b):
        with self.out:
            clear_output()
            exibir_mensagem("Iniciando comparação...")

        self.btn_export.disabled = True

        selected_ui_options = self.select_data.value
        if not selected_ui_options:
            exibir_erro("Por favor, selecione pelo menos um conjunto de dados para comparar.")
            return

        selected_internal_types = [self.DATA_OPTIONS_MAP[opt] for opt in selected_ui_options]
        method = self.radio_method.value

        all_results_summary = []
        all_errors_details = []

        # Para cada tipo de dado selecionado pelo usuário
        for data_type in selected_internal_types:
            # 1. Filtra os records relevantes
            records_to_process = [rec for rec in self.all_records if rec.tipo_dado_y == data_type]

            # 2. Agrupa por Energia de Ativação
            records_by_ea = defaultdict(list)
            for rec in records_to_process:
                records_by_ea[rec.Ea].append(rec)

            errors_for_this_type = {}
            # 3. Calcula o erro para cada Ea
            for Ea, group in records_by_ea.items():
                if len(group) < 2: continue # Precisa de pelo menos 2 ensaios para comparar

                # Extrai os dados (arrays de [x,y]) de cada record no grupo
                curves_data = [rec.df.dropna().values for rec in group]

                error_value = np.nan
                try:
                    if method == "mrs":
                        error_value = self._calculate_mrs_for_ea(curves_data)
                    elif method == "mpcd":
                        error_value = self._calculate_mpcd_for_ea(curves_data)

                    if pd.notna(error_value) and np.isfinite(error_value):
                        errors_for_this_type[Ea] = error_value
                        all_errors_details.append({
                            "DataType": data_type, "Method": method.upper(),
                            "Ea_kJ_mol": Ea, "Error": error_value
                        })
                except Exception as e:
                    exibir_erro(f"Erro ao calcular {method.upper()} para {data_type} com Ea={Ea}: {e}")

            # 4. Encontra o melhor Ea para este tipo de dado
            if errors_for_this_type:
                best_ea = min(errors_for_this_type, key=errors_for_this_type.get)
                min_error = errors_for_this_type[best_ea]
                all_results_summary.append({
                    "DataType": data_type, "Method": method.upper(),
                    "Best_Ea_kJ_mol": best_ea, "Min_Error": min_error
                })

        # 5. Apresenta os resultados
        if all_results_summary:
            self.results_df = pd.DataFrame(all_results_summary)
            self.comparison_details_df = pd.DataFrame(all_errors_details)

            exibir_mensagem("--- MELHORES ENERGIAS DE ATIVAÇÃO ENCONTRADAS ---")
            display(self.results_df.style.format({'Best_Ea_kJ_mol': '{:.2f}', 'Min_Error': '{:.6g}'}))

            exibir_mensagem("\n--- Detalhamento do Erro vs. Ea ---")
            display(self.comparison_details_df.style.format({'Ea_kJ_mol': '{:.2f}', 'Error': '{:.6g}'}))

            self.btn_export.disabled = False
        else:
            exibir_erro("Não foi possível calcular nenhum resultado. Verifique os dados de entrada e a seleção.")

 # ==========================================================
# MÉTODO CORRIGIDO para a classe Modulo5_4_1Comparisons
# ==========================================================

    def _calculate_mrs_for_ea(self, list_of_xy_arrays: List[np.ndarray]) -> float:
        """
        Calcula o Mean Residual Square (MRS) para um conjunto de curvas,
        operando apenas na faixa de sobreposição dos dados.
        """
        if len(list_of_xy_arrays) < 2:
            return np.nan

        # --- NOVA LÓGICA PARA ENCONTRAR A FAIXA DE SOBREPOSIÇÃO ---
        min_vals = []
        max_vals = []

        # 1. Encontra o mínimo e máximo do eixo Y (densidade/parâmetro) para cada curva
        for xy_array in list_of_xy_arrays:
            # O eixo Y é a segunda coluna (índice 1)
            if xy_array.shape[0] > 0:
                min_vals.append(np.nanmin(xy_array[:, 1]))
                max_vals.append(np.nanmax(xy_array[:, 1]))

        if not min_vals or not max_vals:
            return np.nan # Não há dados válidos

        # 2. A sobreposição começa no maior dos mínimos e termina no menor dos máximos
        overlapping_min = np.max(min_vals)
        overlapping_max = np.min(max_vals)

        # 3. Se não houver sobreposição, não é possível calcular o MRS
        if overlapping_min >= overlapping_max:
            return np.nan

        # 4. Cria o eixo Y comum APENAS na faixa de sobreposição
        common_y_axis = np.linspace(overlapping_min, overlapping_max, 200)
        # --- FIM DA NOVA LÓGICA ---

        interpolated_x_matrix = []
        for xy_array in list_of_xy_arrays:
            # Para MRS, interpolamos x = f(y)
            df_sorted = pd.DataFrame(xy_array, columns=['x','y']).sort_values('y').drop_duplicates('y')
            if len(df_sorted) < 2: continue

            f_interp = interp1d(df_sorted['y'], df_sorted['x'], kind='linear', bounds_error=False, fill_value=np.nan)
            interpolated_x_matrix.append(f_interp(common_y_axis))

        if len(interpolated_x_matrix) < len(list_of_xy_arrays):
            return np.nan # Alguma interpolação falhou

        # Calcula a média e o desvio padrão na direção X (logtheta)
        x_matrix = np.array(interpolated_x_matrix)

        # Com a nova lógica, não devemos ter colunas inteiras de NaN
        mean_x_at_y = np.nanmean(x_matrix, axis=0)

        squared_errors = (x_matrix - mean_x_at_y)**2
        mrs = np.nanmean(squared_errors)

        return mrs

   # ==========================================================
# MÉTODO CORRIGIDO para a classe Modulo5_4_1Comparisons
# ==========================================================

    def _calculate_mpcd_for_ea(self, list_of_xy_arrays: List[np.ndarray]) -> float:
        """
        Calcula o Mean Perpendicular Curve Distance (MPCD), operando apenas
        na faixa de sobreposição do eixo X (logtheta).
        """
        if len(list_of_xy_arrays) < 2:
            return np.nan

        # --- NOVA LÓGICA PARA ENCONTRAR A FAIXA DE SOBREPOSIÇÃO DO EIXO X ---
        min_vals = []
        max_vals = []

        # 1. Encontra o mínimo e máximo do eixo X (logtheta) para cada curva
        for xy_array in list_of_xy_arrays:
            # O eixo X é a primeira coluna (índice 0)
            if xy_array.shape[0] > 0:
                min_vals.append(np.nanmin(xy_array[:, 0]))
                max_vals.append(np.nanmax(xy_array[:, 0]))

        if not min_vals or not max_vals:
            return np.nan # Não há dados válidos

        # 2. A sobreposição começa no maior dos mínimos e termina no menor dos máximos
        overlapping_min = np.max(min_vals)
        overlapping_max = np.min(max_vals)

        # 3. Se não houver sobreposição, não é possível calcular o MPCD
        if overlapping_min >= overlapping_max:
            return np.nan

        # 4. Cria o eixo X comum APENAS na faixa de sobreposição
        common_x_axis = np.linspace(overlapping_min, overlapping_max, 200)
        # --- FIM DA NOVA LÓGICA ---

        # O resto da função continua, mas agora usando o 'common_x_axis' seguro
        interpolated_y_matrix = []
        for xy_array in list_of_xy_arrays:
            df_sorted = pd.DataFrame(xy_array, columns=['x','y']).sort_values('x').drop_duplicates('x')
            if len(df_sorted) < 2: continue
            f_interp = interp1d(df_sorted['x'], df_sorted['y'], kind='linear', bounds_error=False, fill_value=np.nan)
            interpolated_y_matrix.append(f_interp(common_x_axis))

        if len(interpolated_y_matrix) < len(list_of_xy_arrays): return np.nan

        y_matrix = np.array(interpolated_y_matrix)
        avg_y_curve = np.nanmean(y_matrix, axis=0)

        valid_avg_mask = ~np.isnan(avg_y_curve)
        if np.sum(valid_avg_mask) < 2: return np.nan

        f_avg = interp1d(common_x_axis[valid_avg_mask], avg_y_curve[valid_avg_mask], kind='cubic', bounds_error=False, fill_value="extrapolate")

        all_perp_distances = []
        for xy_array in list_of_xy_arrays:
            for point in xy_array:
                # Apenas calcula a distância para pontos dentro da faixa de sobreposição
                if overlapping_min <= point[0] <= overlapping_max:
                    def distance_sq(x_c):
                        return (point[0] - x_c)**2 + (point[1] - f_avg(x_c))**2

                    res = minimize_scalar(distance_sq, bounds=(overlapping_min, overlapping_max), method='bounded')
                    if res.success:
                        all_perp_distances.append(np.sqrt(res.fun))

        return np.mean(all_perp_distances) if all_perp_distances else np.nan
# =================================================================================
# MÉTODO CORRIGIDO para a classe Modulo5_4_1Comparisons
# =================================================================================
# ========================================================================
# ADICIONE ESTE MÉTODO DENTRO DA CLASSE Modulo5_4_1Comparisons
# ========================================================================

    def _on_back_clicked(self, b):
        """
        Callback para o botão 'Voltar'.
        Chama o método display do módulo pai (o roteador) para retornar.
        """
        with self.out:
            clear_output()

        if self.parent and hasattr(self.parent, 'display'):
            exibir_mensagem("Retornando à tela de seleção de método...")
            self.parent.display()
        else:
            exibir_erro("Controlador (parent) ou método de retorno não encontrado.")

    def _on_export(self, b):
        with self.out:
            clear_output()

        # CORREÇÃO APLICADA NESTA LINHA:
        if self.results_df is None or self.comparison_details_df is None:
            exibir_erro("Nenhum resultado para exportar. Execute a comparação primeiro.")
            return

        uid = uuid4().hex[:6]
        fname = f"MSC_Comparison_Results_{uid}.xlsx"
        # Usando a forma de caminho recomendada para funcionar em múltiplos sistemas
        path = Path("/tmp") / fname

        try:
            with pd.ExcelWriter(path, engine="openpyxl") as writer:
                # DataFrame com o resumo dos melhores resultados
                self.results_df.to_excel(writer, sheet_name="Best_Ea_Summary", index=False)

                # DataFrame com o detalhamento de todos os erros calculados
                self.comparison_details_df.to_excel(writer, sheet_name="All_Errors_vs_Ea", index=False)

            # Gera o link para download
            link = gerar_link_download(self.results_df, nome_arquivo=fname) # Apenas para gerar o link, o arquivo já está salvo.
            display(link)
            exibir_mensagem(f"Exportação concluída. Arquivo '{fname}' salvo.")

        except Exception as e:
            exibir_erro(f"Ocorreu um erro durante a exportação: {e}")

In [ ]:
# modulo5_4_2_Ref.py (VERSÃO FINAL AJUSTADA)

import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
from typing import List
from collections import defaultdict

# Importações necessárias
# from core import SinteringDataRecord
# from modulo1_interface import exibir_mensagem, exibir_erro, boltzmann_sigmoid

class Modulo5_4_2Ref:
    """
    Submódulo 5.4.2 Refatorado – Método de Erro de Ajuste Individual (MSE).

    Encontra a melhor Energia de Ativação (Ea) para cada curva individualmente,
    identificando qual valor de Ea, dentre os já calculados, resulta em uma curva
    com o menor erro quadrático médio (MSE) ao ser ajustada por uma função sigmoidal.
    """
    def __init__(self, sintering_records: List[SinteringDataRecord], parent=None):
        self.all_records = sintering_records
        self.parent = parent
        self.results_df = None
        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        # Mapeamento da UI para os tipos de dados internos
        self.DATA_OPTIONS_MAP = {
            "Densidade Alinhada": "dados_alinhados",
            "Parâmetro Psi (ψ)": "psi",
            "Parâmetro Phi (φ)": "phi",
            "Densidade Original": "densidade_original"
        }

        # Filtra as opções da UI para mostrar apenas os tipos de dados que existem nos records
        available_types = set(rec.tipo_dado_y for rec in self.all_records)
        self.available_options = [ui_text for ui_text, internal_type in self.DATA_OPTIONS_MAP.items() if internal_type in available_types]

        if not self.available_options:
            self.main_ui = widgets.HTML("<b>Nenhum tipo de dado compatível com este método foi encontrado.</b>")
            return

        self.select_data = widgets.Dropdown(
            options=self.available_options,
            description="Analisar dados de:",
            style={'description_width': 'initial'}
        )

        self.btn_calc = widgets.Button(description="Calcular Melhor Ea (por Erro Individual)", button_style='success')
        self.btn_calc.on_click(self._on_calc)

        self.btn_back = widgets.Button(description="Voltar", button_style='warning')
        self.btn_back.on_click(self._on_back_clicked)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.4.2 – Método de Erro de Ajuste Individual</h3>"),
            widgets.HTML("<p>Este método avalia qual das Energias de Ativação já calculadas produz a curva mais 'perfeita' (com menor erro de auto-ajuste) para cada ensaio.</p>"),
            self.select_data,
            widgets.HBox([self.btn_back, self.btn_calc]),
            self.out
        ])

    def display(self):
        """Exibe a interface principal do módulo."""
        display(self.main_ui)

    def _calc_mse_error(self, x_vals, y_vals):
        """Calcula o erro quadrático médio (MSE) de um ajuste Boltzmann."""
        if len(x_vals) < 4: return np.inf
        try:
            # Estimativas iniciais para os parâmetros do ajuste
            p0 = [np.max(y_vals), np.min(y_vals), np.median(x_vals), (np.max(x_vals) - np.min(x_vals)) / 4]
            popt, _ = curve_fit(boltzmann_sigmoid, x_vals, y_vals, p0=p0, maxfev=5000)
            y_pred = boltzmann_sigmoid(x_vals, *popt)
            return np.mean((y_vals - y_pred)**2)
        except Exception:
            return np.inf

    def _on_calc(self, b):
        with self.out:
            clear_output()

        selected_type = self.DATA_OPTIONS_MAP[self.select_data.value]

        records_to_process = [rec for rec in self.all_records if rec.tipo_dado_y == selected_type]
        if not records_to_process:
            exibir_erro(f"Nenhum dado do tipo '{selected_type}' foi encontrado para análise.")
            return

        # Agrupa os resultados por ensaio, pois a análise é individual
        results_by_ensaio = defaultdict(dict)
        for rec in records_to_process:
            df_clean = rec.df.dropna()

            # Determina qual coluna é o eixo X (logtheta) dinamicamente
            x_col = next((col for col in df_clean.columns if 'logtheta' in col), None)
            y_col = next((col for col in df_clean.columns if 'valor' in col), None)

            if x_col is None or y_col is None: continue

            error = self._calc_mse_error(df_clean[x_col].values, df_clean[y_col].values)

            if np.isfinite(error):
                results_by_ensaio[rec.ensaio_id][rec.Ea] = error

        # Encontra a melhor Ea para cada ensaio e formata o resultado
        summary_list = []
        for ensaio_id, ea_errors in results_by_ensaio.items():
            if not ea_errors: continue

            best_ea = min(ea_errors, key=ea_errors.get)
            min_error = ea_errors[best_ea]

            summary_list.append({
                "Ensaio_ID": ensaio_id + 1,
                "Tipo de Dado": selected_type,
                "Melhor_Ea (kJ/mol)": best_ea,
                "Min_MSE": min_error
            })

        if summary_list:
            self.results_df = pd.DataFrame(summary_list)
            exibir_mensagem("--- MELHOR Ea POR ENSAIO (MÉTODO DE ERRO INDIVIDUAL) ---")
            display(self.results_df.style.format({'Melhor_Ea (kJ/mol)': '{:.2f}', 'Min_MSE': '{:.6g}'}))
        else:
            exibir_erro("Não foi possível calcular nenhum resultado válido.")

    def _on_back_clicked(self, b):
        """Retorna à tela do roteador."""
        if self.parent:
            self.parent.display()

In [ ]:
# modulo5_4_3_blaine_linear.py (VERSÃO REFATORADA)

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
from scipy.stats import linregress
import matplotlib.pyplot as plt
from typing import List

# Importações necessárias
# from core import SinteringDataRecord
# ...

class Modulo5_4_3BlaineLinear:
    """
    Submódulo 5.4.3 Refatorado – Método de Linearização de Blaine.
    Aplica uma transformação linearizante a dados de 'psi' ou 'phi' e
    realiza uma regressão linear para avaliar a qualidade do ajuste (R²).
    """
    def __init__(self, sintering_records: List[SinteringDataRecord], parent=None):
        self.all_records = sintering_records
        self.parent = parent
        self.results_df = None
        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        self.select_param = widgets.Dropdown(
            options=["Parâmetro Psi (ψ)", "Parâmetro Phi (φ)"],
            description="Linearizar:",
        )
        self.btn_linearize = widgets.Button(description="Executar Linearização", button_style='success')
        self.btn_linearize.on_click(self._on_linearize)

        self.btn_plot = widgets.Button(description="Visualizar Linearização", button_style='warning')
        self.btn_plot.on_click(self._on_plot)

        self.btn_back = widgets.Button(description="Voltar", button_style='warning')
        self.btn_back.on_click(self._on_back_clicked)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.4.3 – Método de Linearização</h3>"),
            self.select_param,
            widgets.HBox([self.btn_linearize, self.btn_plot, self.btn_back]),
            self.out
        ])

    def display(self):
        display(self.main_ui)

    def _on_linearize(self, b):
        with self.out:
            clear_output()

        param_choice = 'psi' if "Psi" in self.select_param.value else 'phi'
        records_to_process = [rec for rec in self.all_records if rec.tipo_dado_y == param_choice]

        if not records_to_process:
            exibir_erro(f"Nenhum dado do tipo '{param_choice}' encontrado para linearizar.")
            return

        results_list = []
        for rec in records_to_process:
            df = rec.df.dropna()
            if len(df) < 2: continue

            x_vals = df['logtheta_alinhado'].values
            y_vals = df['valor'].values
            y_transformed = np.nan

            try:
                if param_choice == 'psi':
                    # Transf: ln(y / (1 - y)) - garante que y esteja entre 0 e 1
                    valid_mask = (y_vals > 1e-9) & (y_vals < 1 - 1e-9)
                    if np.sum(valid_mask) < 2: continue
                    y_transformed = np.log(y_vals[valid_mask] / (1 - y_vals[valid_mask]))
                    x_transformed = x_vals[valid_mask]

                elif param_choice == 'phi':
                    # Transf: ln(y) - garante que y > 0
                    valid_mask = y_vals > 1e-9
                    if np.sum(valid_mask) < 2: continue
                    y_transformed = np.log(y_vals[valid_mask])
                    x_transformed = x_vals[valid_mask]

                # Regressão Linear
                slope, intercept, r_value, p_value, std_err = linregress(x_transformed, y_transformed)

                results_list.append({
                    "Ensaio_ID": rec.ensaio_id + 1, "Ea_kJ_mol": rec.Ea,
                    "DataType": param_choice, "R_squared": r_value**2,
                    "Slope": slope, "Intercept": intercept
                })
            except Exception as e:
                exibir_erro(f"Erro na linearização para Ensaio {rec.ensaio_id+1}, Ea={rec.Ea}: {e}")

        if results_list:
            self.results_df = pd.DataFrame(results_list).sort_values(by="R_squared", ascending=False)
            exibir_mensagem("--- RESULTADOS DA LINEARIZAÇÃO (ORDENADO POR R²) ---")
            display(self.results_df)
        else:
            exibir_erro("Nenhum resultado de linearização pôde ser calculado.")

    # Implemente _on_plot e _on_back_clicked de forma similar aos outros módulos
    def _on_plot(self, b):
        with self.out:
            clear_output()
        exibir_mensagem("Função de plotagem para este módulo ainda a ser implementada.")

    def _on_back_clicked(self, b):
        if self.parent:
            self.parent.display()

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output
import numpy as np
import pandas as pd

class Modulo6_0_Arrhenius:
    """
    Módulo 6.0 – Interface para execução do cálculo de Arrhenius.
    """

    def __init__(self, dfs):
        self.dfs_ensaios = dfs
        self.arrhenius_results = {}

        self.btn_calc = widgets.Button(description="Calcular Arrhenius", button_style='success')
        self.btn_export = widgets.Button(description="Exportar Resultados Arrhenius", button_style='info')
        self.out = widgets.Output()

        self.btn_calc.on_click(self._on_calc)
        self.btn_export.on_click(self._on_export)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Módulo 6.0 – Cálculo de Arrhenius</h3>"),
            self.btn_calc,
            self.btn_export,
            self.out
        ])

    def _on_calc(self, b):
        with self.out:
            clear_output()
            exibir_mensagem("[INFO] Iniciando os Cálculos para Curvas de Arrhenius")

        self.arrhenius_results = {}  # limpa antigo

        for i, df in enumerate(self.dfs_ensaios):
            # Identifica colunas
            time_col = next((col for col in df.columns if "Time_s" in col), None)
            temp_col = next((col for col in df.columns if "Temperature_C" in col), None)
            dens_col = next((col for col in df.columns if "DensidadePct" in col), None)


            if not (time_col and temp_col and dens_col):
                with self.out:
                    exibir_erro(f"[ERRO] Ensaio {i+1}: Faltam as colunas necessárias. Pulando.")
                continue

            df_s = df.sort_values(dens_col).reset_index(drop=True)
            dens_min = df_s[dens_col].min()
            dens_max = df_s[dens_col].max()

            if dens_max <= dens_min:
                with self.out:
                    exibir_erro(f"[AVISO] Ensaio {i+1}: Densidade Constante ({dens_min}). Pulando.")
                continue

            n_points = 5
            dens_targets = [
                dens_min + idx_*(dens_max - dens_min)/(n_points - 1)
                for idx_ in range(n_points)
            ]

            try:
                T_K_s = df_s[temp_col] + 273.15
                dTdt_s = np.gradient(T_K_s, df_s[time_col])
                dpdT_s = np.gradient(df_s[dens_col], T_K_s)
            except Exception as e:
                with self.out:
                    exibir_erro(f"[ERRO] Ensaio {i+1}: Falha no cálculo dos gradientes - {e}")
                continue

            from scipy.interpolate import interp1d
            try:
                f_T = interp1d(df_s[dens_col], T_K_s, bounds_error=False, fill_value="extrapolate")
                f_dTdt = interp1d(df_s[dens_col], dTdt_s, bounds_error=False, fill_value="extrapolate")
                f_dpdT = interp1d(df_s[dens_col], dpdT_s, bounds_error=False, fill_value="extrapolate")
            except Exception as e:
                with self.out:
                    exibir_erro(f"[ERRO] Ensaio {i+1}: Falha na criação dos interpoladores - {e}")
                continue

            results_ensaio = []
            for dens in dens_targets:
                if dens < df_s[dens_col].min() or dens > df_s[dens_col].max():
                    continue
                try:
                    T_val = f_T(dens)
                    dTdt_val = f_dTdt(dens)
                    dpdT_val = f_dpdT(dens)
                except Exception as e:
                    with self.out:
                        exibir_erro(f"[ERRO] Ensaio {i+1}: Erro ao interpolar p/ dens={dens}: {e}")
                    continue

                if T_val <= 0:
                    continue

                X = T_val * dpdT_val * dTdt_val
                if X <= 0:
                    continue

                try:
                    ln_val = np.log(X)
                except Exception as e:
                    with self.out:
                        exibir_erro(f"[ERRO] Ensaio {i+1}: Erro ao calcular ln(X) p/ dens={dens}: {e}")
                    continue

                inv_T = 1.0 / T_val
                results_ensaio.append((dens, inv_T, ln_val))

            self.arrhenius_results[i] = results_ensaio

        with self.out:
            exibir_mensagem("[INFO] Parâmetros para Curvas de Arrhenius calculados com 5 densidades igualmente espaçadas. Use Plotar para visualizar.")


    def _on_export(self, b):
        with self.out:
            clear_output()

            # 1) Verifica se há resultados
            if not self.arrhenius_results or not any(self.arrhenius_results.values()):
                exibir_erro("[ERRO] Nenhum resultado disponível para exportar. Execute o cálculo primeiro.")
                return

            exibir_mensagem("[INFO] Preparando exportação dos resultados de Arrhenius...")

            # 2) Monta o DataFrame unificado
            df_all = []
            for i, tup_list in self.arrhenius_results.items():
                if not tup_list:
                    continue
                df_temp = pd.DataFrame(tup_list, columns=["DensidadePct", "InvT", "lnX"])
                df_temp["Ensaio"] = i + 1
                df_all.append(df_temp)

            if not df_all:
                exibir_erro("[ERRO] Não há dados de Arrhenius para exportar.")
                return

            df_export = pd.concat(df_all, ignore_index=True)

            # 3) Usa gerar_link_download para escrever em /tmp e exibir o link
            link = gerar_link_download(
                df_export,
                nome_arquivo="Arrhenius_Results.xlsx"
            )
            display(link)

            exibir_mensagem("[INFO] Exportação Arrhenius concluída com sucesso!")


    def get_results(self):
        """
        Retorna o dicionário arrhenius_results calculado.
        """
        return self.arrhenius_results


In [ ]:
# modulo6.1_arrhenius_display.py

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64
from io import BytesIO
from scipy.stats import linregress

# Constante universal dos gases
R = 8.314

class Modulo6_1ArrheniusDisplay:
    """
    Submódulo para Exibição de Curvas Arrhenius e Tabela de Ea
    ----------------------------------------------------------
    1) Recebe arrhenius_results (dict) do passo anterior:
       arrhenius_results[i] = [(dens, inv_T, lnVal), ...]
    2) Combina os pontos de todos os ensaios e agrupa por densidade.
    3) Faz regressão linear ln(...) vs. 1/T para cada densidade.
    4) Exibe gráfico único com todas as densidades e retas,
       e gera tabela 'dens vs Ea'.
    5) Exporta em Excel e tem botão Back.
    """

    def __init__(self, arrhenius_results, n_ensaios):
        """
        :param arrhenius_results: dict => arrhenius_results[i] = list of (dens, inv_T, ln_val)
        :param n_ensaios: quantidade de ensaios. Se <2, método não é válido.
        """
        self.arrhenius_results = arrhenius_results
        self.n_ensaios = n_ensaios

        # Tabela final dens vs Ea
        self.results_df = None

        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        if self.n_ensaios <2:
            self.main_ui = widgets.HTML("<b>Arrhenius method requires at least 2 tests.</b>")
            return

        explanation = """
        <div style="font-family: Arial, sans-serif; line-height: 1.5;">
            <h3>Arrhenius Display</h3>
            <p>
                Esse módulo combina pontos  (ln(...) vs. 1/T)
                de todos os testes, agrupados por densidade. A partir daí é determinada
                a inclinação levando à energia de ativação
                 Ea = -(Inclinação)*R.
            </p>
            <p>
                É exbido um diagrama com todas as tensidades e uma tabela Densidade vs Ea.
            </p>
        </div>
        """
        self.html_explanation = widgets.HTML(value=explanation)

        self.btn_plot = widgets.Button(description="Plotar & Mostrar Tabela", button_style='success')
        self.btn_plot.on_click(self._on_plot)

        self.btn_export = widgets.Button(description="Exportar Ea e Tabela", button_style='info')
        self.btn_export.on_click(self._on_export)

        self.btn_back = widgets.Button(description="Voltar", button_style='warning')
        self.btn_back.on_click(self._on_back_clicked)

        self.main_ui = widgets.VBox([
            self.html_explanation,
            widgets.HBox([self.btn_plot, self.btn_export, self.btn_back]),
            self.out
        ])

    def display(self):
        if hasattr(self, 'main_ui'):
            display(self.main_ui)
        else:
            display(widgets.HTML("<b>No interface for Arrhenius display available.</b>"))

    def _on_plot(self, b):
        with self.out:
            clear_output()

        if not self.arrhenius_results:
            exibir_erro("[ERRO] Nenhum arrhenius_results encontrado. Execute o cálculo primeiro.")
            return

        from collections import defaultdict
        dens_map = defaultdict(list)  # Mapear densidade para lista de (invT, lnVal)

        # Preenche dens_map com tratamento de exceções (caso algum item não tenha o formato esperado)
        for i, arr_list in self.arrhenius_results.items():
            for item in arr_list:
                try:
                    if len(item) != 3:
                        raise ValueError("Item com número incorreto de elementos.")
                    dens, invT, lnVal = item
                    dens_map[dens].append((invT, lnVal))
                except Exception as e:
                    exibir_erro(f"[ERRO] Ensaio {i+1}: Item inválido {item} - {e}")
                    continue

        if not dens_map:
            exibir_erro("[ERRO] dens_map está vazio. Possivelmente não há dados válidos.")
            return

        try:
            fig, ax = plt.subplots(figsize=(6,5))
            ax.set_title("Arrhenius: ln(...) vs 1/T (Combined)")
            ax.set_xlabel("1/T (1/K)")
            ax.set_ylabel("ln(T·dp/dT·dT/dt)")
        except Exception as e:
            exibir_erro(f"[ERRO] Falha na criação do gráfico: {e}")
            return

        colors = ['red', 'blue', 'green', 'orange', 'purple', 'magenta', 'brown', 'gray', 'cyan', 'black']
        color_idx = 0
        results = []

        for dens, points in sorted(dens_map.items()):
            if len(points) < 2:
                continue
            x_vals = np.array([p[0] for p in points])
            y_vals = np.array([p[1] for p in points])
            if len(x_vals) < 2:
                continue
            try:
                slope, intercept, r_val, p_val, std_err = linregress(x_vals, y_vals)
                Ea_calc = -slope * R
            except Exception as e:
                exibir_erro(f"[ERRO] Falha na regressão para densidade {dens}%: {e}")
                continue

            c = colors[color_idx % len(colors)]
            color_idx += 1

            ax.plot(x_vals, y_vals, 'o', color=c, label=f"{dens}%")
            x_fit = np.linspace(np.min(x_vals), np.max(x_vals), 20)
            y_fit = slope * x_fit + intercept
            ax.plot(x_fit, y_fit, '-', color=c, label=f"{dens}% (Ea={Ea_calc:.2f} J/mol)")
            results.append((dens, Ea_calc))

        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax.grid(True)
        plt.tight_layout()
        plt.show()

        if results:
            self.results_df = pd.DataFrame(results, columns=["Densidade(%)", "Ea (J/mol)"])
            display(self.results_df)
        else:
            exibir_erro("[ERRO] Pontos insuficientes para regressão.")


    def _on_export(self, b):
        with self.out:
            clear_output()
            exibir_mensagem("[INFO] Preparando exportação da tabela de Ea...")

            # 1) Verifica se há resultados
            if self.results_df is None or self.results_df.empty:
                exibir_erro("[ERRO] Nenhum resultado disponível para exportação. Execute o plot primeiro.")
                return

            # 2) Gera o arquivo em /tmp e retorna o link HTML
            link = gerar_link_download(
                self.results_df,
                nome_arquivo="ArrheniusEaTable.xlsx"
            )
            display(link)

            exibir_mensagem("[INFO] Exportação da tabela de Ea concluída com sucesso!")



    def _on_back_clicked(self, b):
        with self.out:
            clear_output()
        exibir_mensagem("[INFO] Retornando ao roteador...")
        # Se houver um roteador definido no parent, chama o método display dele
        if self.parent:
            self.parent.display()
        else:
            exibir_erro("[ERRO] Roteador (parent) não definido.")


In [ ]:
# modulo7_0_Datahistory.py
import copy
import datetime

class DataHistory:
    """
    Classe para armazenar o histórico dos DataFrames.

    Cada registro armazena:
      - A cópia profunda do DataFrame (para evitar alterações futuras)
      - O nome do módulo de onde os dados vieram
      - Os nomes das colunas no momento do armazenamento
      - Um timestamp indicando quando o estado foi salvo.
    """
    def __init__(self):
        self.history = []  # Lista para armazenar os registros de histórico

    def push(self, data, module_name):
        """
        Salva uma cópia do DataFrame 'data', juntamente com informações do módulo.

        Args:
            data (DataFrame): O DataFrame a ser armazenado.
            module_name (str): Uma string identificando de qual módulo os dados vieram.
        """
        record = {
            'timestamp': datetime.datetime.now(),
            'module': module_name,
            'columns': list(data.columns),
            'data': copy.deepcopy(data)
        }
        self.history.append(record)
        print(f"[HISTORY] Dados salvos de '{module_name}' às {record['timestamp']} com colunas: {record['columns']}")

    def pop(self):
        """
        Remove e retorna o registro mais recente do histórico.

        Returns:
            dict: O registro removido ou None se o histórico estiver vazio.
        """
        if self.history:
            record = self.history.pop()
            print(f"[HISTORY] Dados removidos do histórico de '{record['module']}'")
            return record
        print("[HISTORY] Histórico vazio!")
        return None

    def peek(self):
        """
        Retorna o registro mais recente sem removê-lo.

        Returns:
            dict: O registro mais recente ou None se o histórico estiver vazio.
        """
        if self.history:
            return self.history[-1]
        return None

    def get_all(self):
        """
        Retorna todos os registros de histórico.

        Returns:
            list: Lista de registros de histórico.
        """
        return self.history

# =============================================================================
# Exemplo de uso:
#
# Suponha que você tenha um DataFrame 'df' após o processamento em um módulo.
#
# from pandas import DataFrame
#
# df = DataFrame({
#     'Time_s': [0, 10, 20],
#     'Temperature_C': [25, 30, 35],
#     'DensidadePct': [50, 55, 60]
# })
#
# # Cria uma instância do histórico (pode ser criado no MainInteractive ou em cada módulo)
# df_history = DataHistory()
#
# # Antes de modificar 'df' (por exemplo, no módulo 5.1), salve o estado atual:
# df_history.push(df, module_name="Module 3: Recorte e Filtros")
#
# # Agora, suponha que 'df' seja modificado pelo módulo 5.1.
# # Mais tarde, se o usuário clicar em "Voltar", você pode restaurar a versão anterior:
# restored_record = df_history.pop()
# if restored_record is not None:
#     df_restored = restored_record['data']
#     print("Restaurado de:", restored_record['module'])
#     print("Colunas:", restored_record['columns'])
#
# # Você também pode obter todo o histórico para, por exemplo, exportar todas as versões:
# all_versions = df_history.get_all()
# for record in all_versions:
#     print(record['module'], record['timestamp'], record['columns'])
#
# =============================================================================


In [ ]:
# ==========================================================
# MainInteractive.py (VERSÃO FINAL REFATORADA)
# ==========================================================
import ipywidgets as widgets
from IPython.display import display, clear_output

# Supondo que as classes dos módulos já foram definidas nas células anteriores
# (Modulo2Importacao, Modulo3Recorte, ModuloLogTheta, Modulo5_1Alinhamento, etc.)

class MainInteractive:
    """
    Controlador principal refatorado.
    Orquestra o fluxo entre os módulos, passando a lista de SinteringDataRecord
    para o fluxo da Curva Mestra (MSC).
    """

    def __init__(self):
        # --- Armazenamento de Estado ---
        # DataFrames brutos
        self.dfs_from_mod2 = None
        self.dfs_from_mod3 = None

        # A nova estrutura de dados que será passada e atualizada no fluxo MSC
        self.sintering_records = []

        # Módulos instanciados
        self.mod2 = None
        self.mod3 = None
        self.logtheta_module = None
        self.alinh_module = None
        self.blaine_module = None
        self.sigmoidal_module = None
        self.roteador_5_4 = None

        # Para o fluxo Arrhenius (não refatorado)
        self.arrhenius_calc_module = None
        self.arrhenius_display_module = None

        self.mod4_choice = None
        self.current_step = 0

        # --- Widgets da UI ---
        self.step_label = widgets.Label("Bem-vindo ao Ogum Sintering!")
        self.top_next = widgets.Button(description="Próxima Etapa", button_style='success')
        self.top_back = widgets.Button(description="Voltar", button_style='warning', disabled=True)
        self.bottom_next = widgets.Button(description="Próxima Etapa", button_style='success')
        self.bottom_back = widgets.Button(description="Voltar", button_style='warning', disabled=True)
        self.log_area = widgets.Output()
        self.modules_container = widgets.VBox([])

        # Conecta callbacks
        self.top_next.on_click(self._on_next_step)
        self.bottom_next.on_click(self._on_next_step)
        self.top_back.on_click(self._on_back_step)
        self.bottom_back.on_click(self._on_back_step)

        # Monta layout principal
        self.main_ui = widgets.VBox([
            widgets.HBox([self.top_back, self.top_next]),
            self.step_label,
            self.log_area,
            self.modules_container,
            widgets.HBox([self.bottom_back, self.bottom_next])
        ])

        self._update_nav_buttons()
        # Inicia o fluxo na tela de introdução
        self._show_intro()

    def display(self):
        display(self.main_ui)

    def _update_nav_buttons(self):
        self.top_back.disabled = (self.current_step <= 1)
        self.bottom_back.disabled = (self.current_step <= 1)

    def _show_intro(self):
        """Exibe a tela inicial."""
        self.current_step = 0 # Define o passo como 0 para a introdução
        intro_text = """
        <h1>Ogum Sintering</h1>
        <p>Aplicação para análise de dados de sinterização. Clique em "Próxima Etapa" para começar.</p>
        <p>
            Este assistente interativo guia você pela importação, limpeza e análise de dados de sinterização,
            permitindo a construção de Curvas Mestras e a análise por meio do método de Arrhenius.
        </p>
        """
        self.modules_container.children = (widgets.HTML(value=intro_text),)
        self.step_label.value = "Passo 0: Introdução"
        self._update_nav_buttons()

    def _on_next_step(self, b):
        self.log_area.clear_output()

        # Avança o estado
        self.current_step += 1

        if self.current_step == 1:
            with self.log_area: exibir_mensagem("=== Módulo 2: Importação e Mapeamento ===")
            self.mod2 = Modulo2Importacao()
            self.modules_container.children = (self.mod2.main_ui,)
            self.step_label.value = "Passo 1: Importe e mapeie seus dados."

        elif self.current_step == 2:
            self.dfs_from_mod2 = self.mod2.get_dfs()
            if not any(df is not None and not df.empty for df in self.dfs_from_mod2):
                exibir_erro("Nenhum dado válido importado do Módulo 2. Impossível prosseguir.")
                self.current_step -= 1 # Permanece no passo atual
                return
            with self.log_area: exibir_mensagem("=== Módulo 3: Recorte e Filtragem ===")
            self.mod3 = Modulo3Recorte(self.dfs_from_mod2)
            self.modules_container.children = (self.mod3.main_ui,)
            self.step_label.value = "Passo 2: Filtre e prepare os dados."

        elif self.current_step == 3:
            self.dfs_from_mod3 = self.mod3.get_dfs()
            if not any(df is not None and not df.empty for df in self.dfs_from_mod3):
                exibir_erro("Nenhum dado válido após a filtragem do Módulo 3. Impossível prosseguir.")
                self.current_step -= 1
                return
            with self.log_area: exibir_mensagem("=== Escolha do Método de Análise ===")
            self.radio_m4 = widgets.RadioButtons(options=[("Curva Mestra (LogTheta)", "logtheta"), ("Arrhenius", "arrhenius")], description="Método:")
            self.btn_m4_confirm = widgets.Button(description="Confirmar Escolha", button_style='info')
            self.btn_m4_confirm.on_click(self._on_m4_confirm)
            self.modules_container.children = (widgets.VBox([widgets.HTML("<h3>Escolha o método principal de análise:</h3>"), self.radio_m4, self.btn_m4_confirm]),)
            self.step_label.value = "Passo 3: Escolha o método."

        elif self.current_step == 4:
            if self.mod4_choice == "logtheta":
                with self.log_area: exibir_mensagem("=== Módulo 4: Cálculo de LogTheta ===")
                self.logtheta_module = ModuloLogTheta(self.dfs_from_mod3)
                self.modules_container.children = (self.logtheta_module.ui,)
                self.step_label.value = "Passo 4 [MSC]: Calcule o LogTheta para diferentes Energias de Ativação."
            elif self.mod4_choice == "arrhenius":
                with self.log_area: exibir_mensagem("=== Módulo 6.0: Cálculo para Arrhenius ===")
                self.arrhenius_calc_module = Modulo6_0_Arrhenius(self.dfs_from_mod3)
                self.modules_container.children = (self.arrhenius_calc_module.main_ui,)
                self.step_label.value = "Passo 4 [Arrhenius]: Execute o cálculo."

        elif self.current_step == 5:
            if self.mod4_choice == "logtheta":
                # ALTERAÇÃO: Pega a nova estrutura de dados
                self.sintering_records = self.logtheta_module.get_sintering_records()
                with self.log_area: exibir_mensagem("=== Módulo 5.1: Alinhamento de Curvas ===")
                self.alinh_module = Modulo5_1Alinhamento(self.sintering_records)
                self.modules_container.children = (self.alinh_module.main_ui,)
                self.step_label.value = "Passo 5.1 [MSC]: Alinhe as curvas."
            elif self.mod4_choice == "arrhenius":
                arr_results = self.arrhenius_calc_module.get_results()
                with self.log_area: exibir_mensagem("=== Módulo 6.1: Análise de Arrhenius ===")
                self.arrhenius_display_module = Modulo6_1ArrheniusDisplay(arr_results, len(self.dfs_from_mod3))
                self.modules_container.children = (self.arrhenius_display_module.main_ui,)
                self.step_label.value = "Passo 5 [Arrhenius]: Análise final."

        elif self.current_step == 6:
            # ALTERAÇÃO: Passa a lista de records para a próxima etapa
            self.sintering_records = self.alinh_module.get_aligned_records()
            with self.log_area: exibir_mensagem("=== Módulo 5.2: Parâmetros de Densificação (Blaine) ===")
            self.blaine_module = Modulo5_2BlaineParameters(self.sintering_records)
            self.modules_container.children = (self.blaine_module.main_ui,)
            self.step_label.value = "Passo 5.2 [MSC]: Calcule os parâmetros Psi e Phi."

        elif self.current_step == 7:
            # ALTERAÇÃO: Passa a lista de records para a próxima etapa
            self.sintering_records = self.blaine_module.get_blaine_records()
            with self.log_area: exibir_mensagem("=== Módulo 5.3: Ajuste Sigmoidal ===")
            self.sigmoidal_module = Modulo5_3Sigmoides(self.sintering_records)
            self.modules_container.children = (self.sigmoidal_module.main_ui,)
            self.step_label.value = "Passo 5.3 [MSC]: Aplique o ajuste de curva."

        elif self.current_step == 8:
            # ALTERAÇÃO: Passa a lista de records para a próxima etapa
            self.sintering_records = self.sigmoidal_module.get_sigmoidal_records()
            with self.log_area: exibir_mensagem("=== Módulo 5.4: Roteador de Comparações ===")
            self.roteador_5_4 = Modulo5_4_Roteador(self.sintering_records, controller=self)
            self.modules_container.children = [self.roteador_5_4.main_ui]
            self.step_label.value = "Passo 5.4 [MSC]: Escolha o método final para encontrar a melhor Ea."

        else:
            with self.log_area:
                exibir_mensagem("Fim do fluxo de análise.")
            self.modules_container.children = (widgets.HTML("<h2>Análise concluída.</h2><p>Você pode voltar para revisar os passos anteriores.</p>"),)

        self._update_nav_buttons()

    def _on_back_step(self, b):
        self.log_area.clear_output()
        # Simplesmente volta um passo e redesenha a tela
        self.current_step -= 2
        self._on_next_step(None)

    def show_module_by_step(self, step_number):
        """
        Força a navegação para um passo específico do wizard.
        Útil para ser chamado por módulos filhos (ex: botão 'Voltar').
        """
        # Define o passo alvo e chama a lógica de redesenhar a tela
        self.current_step = step_number - 1
        self._on_next_step(None)

    def _on_m4_confirm(self, b):
        """Callback para confirmar a escolha do método e avançar."""
        self.mod4_choice = self.radio_m4.value
        self._on_next_step(None)

# Para iniciar a interface no Notebook:
# Certifique-se de que a classe SinteringDataRecord e todas as classes de módulo
# estejam definidas em células anteriores antes de executar esta.
main_app = MainInteractive()
main_app.display()

[INFO] 3 Testes serão carregados.
[INFO] Ensaio 1: Removidos 8 pontos fora de [0..100]%.
[INFO] Ensaio 1: Interpolação cubic gerou 30 pontos.
[INFO] Ensaio 1: Filtro/Interp 'cubic' finalizado. Tamanho DF=30.
[HISTORY] Dados salvos de 'Module 3: Filtro/Interpolação Test 1' às 2025-07-02 18:53:12.442850 com colunas: ['Time_s_e1', 'DensidadePct_e1', 'Temperature_C_e1']
[INFO] Ensaio 2: Removidos 24 pontos fora de [0..100]%.
[INFO] Ensaio 2: Interpolação cubic gerou 30 pontos.
[INFO] Ensaio 2: Filtro/Interp 'cubic' finalizado. Tamanho DF=30.
[HISTORY] Dados salvos de 'Module 3: Filtro/Interpolação Test 2' às 2025-07-02 18:53:31.176851 com colunas: ['Time_s_e2', 'DensidadePct_e2', 'Temperature_C_e2']
[INFO] Ensaio 3: Removidos 100 pontos fora de [0..100]%.
[INFO] Ensaio 3: Interpolação cubic gerou 30 pontos.
[INFO] Ensaio 3: Filtro/Interp 'cubic' finalizado. Tamanho DF=30.
[HISTORY] Dados salvos de 'Module 3: Filtro/Interpolação Test 3' às 2025-07-02 18:53:43.051412 com colunas: ['Time_s_e3